# Packages

In [ ]:
%pip install transformers torch scikit-learn datasets
%pip install ckip-transformers nlpaug googletrans==4.0.0-rc1

In [ ]:
%matplotlib inline

import os
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Configuration

In [ ]:
from collections import defaultdict

聖意Doc = {
    "交易": ["跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。"],
    "婚姻": ["這是自古以來千年不變的人生大哉問！婚姻這解籤項目，當然也涵蓋了愛情問題，基本上就是親密關係的解答提示。因為人心多變、難測，只能來問神，希望能透過神明的超能力，來指引或是平復各種心情。"],
    "求財": ["這個跟「交易」有什麼差異呢？直白一點講，交易比較是做生意的場景、狀態，有商品買賣，或是合約簽訂，總之是雙方以物易物，或是金錢換東西，交易會賺錢，但是也有可能是換東西或是資源回來，但是「求財」是聚焦在我要錢、給我錢，這個重點上面，所以有可能是會不會彩卷中獎的偏財、突然繼承遺產的橫財，或是股票投資這類性質的問題。求財這個解籤項目，還會有時間訊息提示，比如：秋冬，預告財運亨通的時間區段。"],
    "自身": ["也有一說叫做「身命」，這個就是卜問個人最近的運勢情況了，解答的層面涵蓋很廣，甚至是涵蓋了其他的解籤項目。"],
    "家宅": ["也稱「家運」，或是另一種說法「宅舍」等等之類的，就是只關於家裡大大小小事情的解籤項目，這是預測家中組成份子的命運或是即將發生的事情。"],
    "六畜": ["或說是「納畜」。以前農業社會，除了耕種農作之外，通常家裡也會養雞、養鴨、養豬之類的，可以作為副業貼補家用，或是為了年節祭祀的牲禮需要，因這個畜養是有目的性的，也有相當財力、心力投入的，自然就會擔心會不會打水漂沒有收穫，比如養雞的最怕雞瘟，養豬也是一樣，所以這也是一個具有重要性的占卜問題。不過現代工商社會，多數人養的是寵物，這方面的問題解籤，也歸類到這裡。"],
    "田蠶": ["也稱「耕作」、「農業」。「田蠶」是指種植桑樹及餵養蠶的工作事務，「漢書.地理志下」：殷道衰， 箕子去之朝鮮，教其民以禮義，田蠶織作。還有，「魏書·韓顯宗傳」：田蠶暫廢，則將來無資。此國之深憂也。古代的土地制度是有規劃一塊專門種植桑樹的田，就是為了要養蠶；而中國古代的賦稅制度，更是有專門的「絹稅」；還有在貨幣的制度規定裡，也有絲綢貨幣；過去農業社會，種植桑樹養蠶取絲製綢，是很重要的經濟產物，也是國力國庫來源，因為古時候的中國可是靠絲綢賺取大把大把外匯的，過去中國綢緞是聞名於世的紡織品，絲綢之路就是這麼走出來的。不過但凡是農作物相關，就無法不看老天爺的臉色，天災、蟲禍還是突變的氣候，都會讓辛苦的耕耘，一夕之間變成過眼雲煙。所以農民到寺廟中祈求風調雨順，同時求籤探詢神意，就是想知道，自己的辛苦和希望會不會全都白費。以現今工商社會來看，田蠶這個項目好像已經派不上用場，不過我個人有些觀點想法，提供大家參考，田蠶這個場景雖然與現在社會情況不太符合，但是想要探詢的「神意」基礎卻是相通的，尤其是做自媒體內容創作的人，直白的說就是KOL、網紅，他們的田就是社群網路平台帳號，或是自己網站頻道，他們種得桑樹，就是自己創作出來的各種形式內容，他們細心照料養的蠶，就是點閱流量，而各位粉絲大大們就是「神意」。就像也有別的節目做籤詩的內容，但是我這樣查資料看文獻，像是做學問一樣的在介紹籤詩，就不知道各位粉絲們是不是有鍾意聽，如果我想來求神問卜看看籤詩怎麼說的話，在聖意、解籤項目，我就會看「田蠶」這個項目是怎麼解說的。"],
    "尋人": ["或是說「消息」。這個部分可以看成是「尋人」，比如尋找失散的親人，行蹤不明的人。解籤的關鍵字通常會出現：「見」，見到的見、「至」，到達的至；「遲」，遲到的遲；「難」，困難的難；「杳」，音訊飄渺、杳然的杳。"],
    "行人": ["這個解籤項目有時候會跟「尋人」合併成一個。過去訊息消息傳遞不容易，第一支手機行動電話也就是1973年發明的，再加上網路的科技，才發展到今天的通訊便利，在這之前的更之前，真的是出門就像不見一樣，外地唸書的遊子、四處奔波的商人，或是常需要移動駐紮的軍人，如果杳無音訊，親人感到無助不安的時候，就會想來詢問神明，希望能從中得到這些出外的人是否平安。解籤的關鍵字一般會有：「至」、「動」、「阻」、「有信」，來提供參考。"],
    "六甲": ["就是懷胎六甲的「六甲」，所以這個解籤項目，也稱做「子息」、「產育」。傳宗接代一向是社會價值觀裡，很重要的一環，所以來問懷孕的，或是能不能順利生產的，都在這個項目，當然也會有來問是生男還是生女的，也是看這個解籤項目。解籤的關鍵字有：「安」、「驚」、「有險」。不過，不管籤詩如何回覆您，產檢絕對還是要照時間去做。"],
    "山墳": ["又稱「風水」、「地理」、「宅墳」。風水是大家很普遍有的觀念，除了尋求堪輿老師的協助之外，一般有時候喪葬活動或是遷移墳墓的吉凶，會到寺廟求籤探詢，尤其牽涉到祖先靈寢有變化，是不是能庇蔭子孫，有時候就很可能會起家族糾紛了，所以很多時候來求這個籤，是希望神明來公正裁決。"],
    "公訟": ["也稱「訴訟」、「官司」等。不管是過去還是現在，大家都是對官司、法院這樣的地方還是敬而遠之的，但是如果實在是惹上了官司訴訟的事情，心裡面有負擔恐懼，不知道判決會如何；或是長期官司纏身，就會來請示神明。解籤的關鍵字有：「勝」、「和」、「有理」、「虧」。"],
    "疾病": ["信眾到廟裡拜拜，一定會祈求身體健康，從個人到家人、在乎的人。在醫療不發達的年代，久病難癒實在會令人擔心、不安，所以除了向神明祈求保佑之外，也會想要求的一個解答，或是希望神明能給予一個提示，是不是病能治得好。解籤的字詞多數會使用「安」、「解」、「祈保」、「祈福」等等，或是也會有「欠安」、「難痊」這樣的字詞。不過不會出現「死」這樣的斷語。"],
    "失物": ["找不見的東西。在今天的時空下，不知道神明最常被問要幫忙找的，是不是寵物啊？這個解籤項目的關鍵字，會有：「見」、「難見」，也會有方位的指示，提供信眾去尋找失物。"],
    "移徙": ["也稱「遷徙」、「移居」。主要是問搬家或是搬辦公室、廠房等等。這個是看陽宅的搬遷。解籤項目多是「吉」或是「守舊」來回覆，是否合適搬遷。"],
}

EXAMPLE_DIVINE_MAP = {
    "交易": "遲",
    "婚姻": "合",
    "求財": "頗有",
    "自身": "安",
    "家宅": "祈保",
    "六畜": "利",
    "田蠶": "利",
    "尋人": "難",
    "行人": "有阻",
    "六甲": "女",
    "山墳": "安吉",
    "公訟": "虧",
    "疾病": "設送（必癒）",
    "失物": "西方(註：北方屬水。西方屬金）",
    "移徙": "守舊",
}


DIVINE_LABELS = {
    "交易": ["交易", "買賣", "商業", "貿易", "交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。"],
    "婚姻": ["婚姻", "婚配", "結婚", "夫妻", "婚姻是兩個人結為夫妻的社會制度，通常涉及法律和情感的承諾。"],
    "求財": ["求財", "財運", "財富", "資金", "求財是指追求財富和資金的行為，通常與投資和理財相關。"],
    "自身": ["自身", "身命", "個人", "自我", "自身是指個人的身體和命運，涉及健康和運勢的評估。"],
    "家宅": ["家宅", "家運", "宅舍", "家庭", "家宅是指家庭居住的房屋，涉及家庭運勢和風水。"],
    "六畜": ["六畜", "納畜", "畜牧", "牲畜", "六畜是指家畜，包括牛、馬、羊、豬、狗和雞，與畜牧業相關。"],
    "田蠶": ["田蠶", "耕作", "農業", "農事", "田蠶是指農業活動，包括耕作和養蠶，與農村經濟相關。"],
    "尋人": ["尋人", "人信", "消息", "找人", "尋人是指尋找失蹤或離家的人，通常涉及消息和線索的收集。"],
    "行人": ["行人", "尋人", "旅人", "遊子", "行人是指旅行或外出的人，通常與旅途和歸期相關。"],
    "六甲": ["六甲", "子息", "產育", "生育", "六甲是指生育和子息，涉及懷孕和產育的過程。"],
    "山墳": ["山墳", "風水", "地理", "宅墳", "山墳是指墓地和風水，涉及地理和祖先的安葬。"],
    "公訟": ["公訟", "訟詞", "訴訟", "官司", "公訟是指法律訴訟，涉及官司和法庭的判決。"],
    "疾病": ["疾病", "病痛", "健康", "醫療", "疾病是指身體的病痛，涉及健康和醫療的問題。"],
    "失物": ["失物", "遺失", "丟失", "尋物", "失物是指遺失的物品，涉及尋找和找回的過程。"],
    "移徙": ["移徙", "遷徙", "移居", "搬家", "移徙是指搬家和遷徙，涉及居住地的改變。"],
}

DIVINE_WEIGHTS = {
    "交易": [
        ("交易", 1.0), ("求財", 0.8), ("田蠶", 0.5), ("六畜", 0.4), ("行人", 0.3)
    ],
    "婚姻": [
        ("婚姻", 1.0), ("自身", 0.7), ("家宅", 0.6), ("六甲", 0.5), ("移徙", 0.3)
    ],
    "求財": [
        ("求財", 1.0), ("交易", 0.8), ("田蠶", 0.6), ("六畜", 0.4), ("疾病", 0.2)
    ],
    "自身": [
        ("自身", 1.0), ("婚姻", 0.7), ("家宅", 0.6), ("疾病", 0.5), ("移徙", 0.4)
    ],
    "家宅": [
        ("家宅", 1.0), ("自身", 0.6), ("山墳", 0.5), ("移徙", 0.5), ("六甲", 0.4)
    ],
    "六畜": [
        ("六畜", 1.0), ("田蠶", 0.7), ("求財", 0.4), ("家宅", 0.3), ("疾病", 0.3)
    ],
    "田蠶": [
        ("田蠶", 1.0), ("六畜", 0.7), ("求財", 0.6), ("交易", 0.5), ("家宅", 0.3)
    ],
    "尋人": [
        ("尋人", 1.0), ("行人", 0.8), ("失物", 0.6), ("移徙", 0.4), ("家宅", 0.3)
    ],
    "行人": [
        ("行人", 1.0), ("尋人", 0.8), ("移徙", 0.6), ("交易", 0.3), ("失物", 0.3)
    ],
    "六甲": [
        ("六甲", 1.0), ("婚姻", 0.5), ("家宅", 0.4), ("疾病", 0.4), ("自身", 0.3)
    ],
    "山墳": [
        ("山墳", 1.0), ("家宅", 0.5), ("移徙", 0.4), ("公訟", 0.3), ("自身", 0.2)
    ],
    "公訟": [
        ("公訟", 1.0), ("自身", 0.4), ("家宅", 0.3), ("山墳", 0.3), ("失物", 0.2)
    ],
    "疾病": [
        ("疾病", 1.0), ("自身", 0.5), ("六甲", 0.4), ("家宅", 0.3), ("六畜", 0.3)
    ],
    "失物": [
        ("失物", 1.0), ("尋人", 0.6), ("行人", 0.3), ("家宅", 0.2), ("公訟", 0.2)
    ],
    "移徙": [
        ("移徙", 1.0), ("家宅", 0.5), ("行人", 0.6), ("山墳", 0.4), ("自身", 0.4)
    ]
}

DIVINE_Q = {
    '交易': [
        '商業談判成功機會',
        '買賣交易的吉凶',
        '合同簽訂的風險評估',
        '生意夥伴的可靠性',
        '商業策略的正確性',
        '交易過程中可能的障礙',
        '談判中的對方態度',
        '價格談判的有利時機',
        '商業投資的潛在收益',
        '貿易和商業環境的變化',
    ],
    '婚姻': [
        '感情發展和婚姻前景',
        '尋找合適的婚姻對象',
        '婚姻是否能帶來幸福',
        '兩人感情的相容性',
        '婚姻中可能出現的問題',
        '伴侶的忠誠度',
        '婚姻生活的穩定性',
        '是否適合結婚',
        '婚姻中的磨合和成長',
        '感情和婚姻的命運轉折',
    ],
    '求財': [
        '投資理財的吉凶',
        '商業機會的成功機率',
        '財運起伏和金錢流向',
        '職業收入增長預測',
        '創業風險評估',
        '是否適合進行大額投資',
        '財務困境的解決方案',
        '意外之財的可能性',
        '職業轉型的財務風險',
        '家庭財務穩定性',
    ],
    '自身': [
        '個人事業發展機會',
        '人生方向和人生目標',
        '個人運勢和福祉',
        '內心困惑和心靈成長',
        '職業發展和前景',
        '個人修為和修行進程',
        '性格缺陷和改進方向',
        '精神和心理健康狀況',
        '個人際遇和命運轉折',
        '自我實現和夢想追求',
    ],
    '家宅': [
        '擔心家庭和睦程度',
        '詢問房屋風水是否吉祥',
        '家人健康狀況的憂慮',
        '家庭財運變化預測',
        '子女教育和發展前景',
        '祖先祭祀是否周全',
        '家庭成員之間的人際關係',
        '住宅環境是否安全舒適',
        '家庭未來發展方向',
        '租屋或購屋的吉凶',
    ],
    '六畜': [
        '畜牧業的發展前景',
        '牲畜健康和繁殖',
        '畜牧投資的風險',
        '牲畜飼養的技術',
        '畜牧市場的價格變動',
        '畜牧業的環境影響',
        '牲畜疾病的預防',
        '畜牧業的創新和轉型',
        '畜牧業補貼和政策',
        '不同品種牲畜的選擇',
    ],
    '田蠶': [
        '農作物收成的預測',
        '農業投資的風險',
        '農場或土地的產能',
        '農業環境的變化',
        '養蠶業的發展前景',
        '農業技術的創新',
        '農產品市場的價格變動',
        '農業補貼和政策',
        '天氣對農業的影響',
        '農業轉型的可能性',
    ],
    '尋人': [
        ' 尋找失散家人',
        ' 追蹤失聯朋友',
        ' 找到商業夥伴',
        ' 尋找潛在的感情對象',
        ' 追查債務人',
        ' 定位親戚或朋友',
        ' 尋找繼承人',
        ' 追蹤遺失的寵物',
        ' 尋找工作夥伴',
        ' 追查家族歷史線索',
    ],
    '行人': [
        '旅行的吉凶',
        '出差或遷徙的運勢',
        '旅行路線的安全性',
        '出國工作或學習的機會',
        '旅行中可能遇到的困難',
        '交通工具的選擇',
        '遠距離旅行的風險',
        '旅行目的地的宜居性',
        '旅行帶來的機遇',
        '國際旅行的文化適應',
    ],
    '六甲': [
        '生育能力和健康',
        '懷孕的可能性',
        '胎兒的健康和發育',
        '生育的吉凶',
        '順利生產的機會',
        '育兒的挑戰和準備',
        '家庭擴大後的調適',
        '生育年齡的適宜性',
        '生育對事業的影響',
        '子女未來的發展潛力',
    ],
    '山墳': [
        ' 祖先墳墓的風水',
        ' 祭祀儀式的完善',
        ' 墳墓位置的吉凶',
        ' 祖先靈位的安置',
        ' 家族墳地的管理',
        ' 祖先牌位的擺設',
        ' 陰陽兩界的聯繫',
        ' 風水對家族運勢的影響',
        ' 祖先崇拜的文化意義',
        ' 墳墓遷移的考慮',
    ],
    '公訟': [
        ' 法律訴訟的勝算',
        ' 訴訟過程中的風險',
        ' 法律糾紛的解決方案',
        ' 訴訟對個人的影響',
        ' 尋找法律支持',
        ' 評估訴訟成本',
        ' 調解和和解的可能性',
        ' 訴訟對名譽的影響',
        ' 法律程序的進展',
        ' 司法判決的公正性',
    ],
    '疾病': [
        ' 健康狀況的預測',
        ' 疾病康復的可能性',
        ' 醫療治療的效果',
        ' 預防疾病的方法',
        ' 慢性病的管理',
        ' 醫療費用的經濟負擔',
        ' 遺傳疾病的風險',
        ' 心理健康和壓力',
        ' 替代療法的選擇',
        ' 疾病對生活和工作的影響',
    ],
    '失物': [
        ' 遺失物品的尋找',
        ' 遺失物品的可能去向',
        ' 尋回遺失物品的機會',
        ' 防止物品遺失的方法',
        ' 遺失物品的賠償',
        ' 追查遺失物品的線索',
        ' 遺失物品對生活的影響',
        ' 保險理賠的可能性',
        ' 重要文件遺失的補救',
        ' 物品遺失的心理影響',
    ],
    '移徙': [
        ' 搬家的吉凶',
        ' 遷徙的適當時機',
        ' 新居住地的宜居性',
        ' 搬家對家庭的影響',
        ' 遷徙的經濟可行性',
        ' 搬家後的社會適應',
        ' 遷徙對事業的影響',
        ' 不同地區的生活成本',
        ' 遷徙的文化衝擊',
        ' 新環境的機會和挑戰',
    ],
}


DIVINE_A = {
    "交易": ["跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。",
           '預示商業或交易的機會',
           '提醒交易中需要謹慎',
           '暗示可能有成功的商業往來',
           '建議仔細評估交易風險',
           '預示可能有重要的商業決策',
           '提示保持誠信和透明',
           '意味著交易可能需要談判',
           '建議做好充分的準備',
           '預示可能有有利的合作機會',
           '提醒注意合同和法律細節',
           ],
    "婚姻": ["這是自古以來千年不變的人生大哉問！婚姻這解籤項目，當然也涵蓋了愛情問題，基本上就是親密關係的解答提示。因為人心多變、難測，只能來問神，希望能透過神明的超能力，來指引或是平復各種心情。",
           '預示婚姻關係的變化',
           '提示感情和婚姻的發展',
           '暗示可能遇到理想伴侶',
           '建議珍惜感情和溝通',
           '預示婚姻中的和諧與理解',
           '提醒處理感情中的挑戰',
           '意味著可能有婚姻契機',
           '建議保持開放和包容的態度',
           '預示感情生活的轉機',
           '提示平衡個人需求和伴侶關係',
           ],
    "求財": ["這個跟「交易」有什麼差異呢？直白一點講，交易比較是做生意的場景、狀態，有商品買賣，或是合約簽訂，總之是雙方以物易物，或是金錢換東西，交易會賺錢，但是也有可能是換東西或是資源回來，但是「求財」是聚焦在我要錢、給我錢，這個重點上面，所以有可能是會不會彩卷中獎的偏財、突然繼承遺產的橫財，或是股票投資這類性質的問題。求財這個解籤項目，還會有時間訊息提示，比如：秋冬，預告財運亨通的時間區段。",
           '預示財運可能有所好轉',
           '提醒審慎投資和理財',
           '暗示可能出現意外財富',
           '建議開拓新的財富機會',
           '預示商業或職業發展',
           '提示避免不必要的金錢風險',
           '意味著財運可能需要耐心等待',
           '建議保持良好的財務規劃',
           '預示可能有意外收入',
           '提醒珍惜和管理現有資源',
           ],
    "自身": ["也有一說叫做「身命」，這個就是卜問個人最近的運勢情況了，解答的層面涵蓋很廣，甚至是涵蓋了其他的解籤項目。",
           '反映個人當前的身心狀態',
           '預示個人運勢和個人發展的方向',
           '提示個人需要注意身體健康',
           '暗示個人修養和成長的機會',
           '展示個人能力和潛力的發揮',
           '建議關注自我提升和學習',
           '預示個人運勢可能有轉機',
           '提醒保持積極正面的心態',
           '意味著個人將面臨重要的選擇',
           '顯示個人在精神和物質層面的平衡',
           ],
    "家宅": ["也稱「家運」，或是另一種說法「宅舍」等等之類的，就是只關於家裡大大小小事情的解籤項目，這是預測家中組成份子的命運或是即將發生的事情。",
           '家宅吉祥，房屋穩固，家庭和睦安定',
           '預示家庭環境將會更加寧靜舒適',
           '表示住宅可能會有改善或擴建',
           '家人關係將更加融洽和諧',
           '提醒注意家宅風水和居住環境',
           '暗示可能會遷入新居或進行裝修',
           '家庭成員之間將互相扶持',
           '預示家庭財運和運勢將會提升',
           '建議保持家居整潔和舒適',
           '意味著家庭將成為個人安全和支持的堡壘',
           ],
    "六畜": ["或說是「納畜」。以前農業社會，除了耕種農作之外，通常家裡也會養雞、養鴨、養豬之類的，可以作為副業貼補家用，或是為了年節祭祀的牲禮需要，因這個畜養是有目的性的，也有相當財力、心力投入的，自然就會擔心會不會打水漂沒有收穫，比如養雞的最怕雞瘟，養豬也是一樣，所以這也是一個具有重要性的占卜問題。不過現代工商社會，多數人養的是寵物，這方面的問題解籤，也歸類到這裡。",
           '預示畜牧業的運勢',
           '提示養殖動物的健康',
           '暗示畜牧業可能有收穫',
           '建議careful管理畜群',
           '預示畜牧業投資可能成功',
           '提醒注意動物疾病預防',
           '意味著可能有新的畜牧機會',
           '建議學習現代畜牧技術',
           '預示畜牧收入可能增加',
           '提示善待動物',
           ],
    "田蠶": ["也稱「耕作」、「農業」。「田蠶」是指種植桑樹及餵養蠶的工作事務，「漢書.地理志下」：殷道衰， 箕子去之朝鮮，教其民以禮義，田蠶織作。還有，「魏書·韓顯宗傳」：田蠶暫廢，則將來無資。此國之深憂也。古代的土地制度是有規劃一塊專門種植桑樹的田，就是為了要養蠶；而中國古代的賦稅制度，更是有專門的「絹稅」；還有在貨幣的制度規定裡，也有絲綢貨幣；過去農業社會，種植桑樹養蠶取絲製綢，是很重要的經濟產物，也是國力國庫來源，因為古時候的中國可是靠絲綢賺取大把大把外匯的，過去中國綢緞是聞名於世的紡織品，絲綢之路就是這麼走出來的。不過但凡是農作物相關，就無法不看老天爺的臉色，天災、蟲禍還是突變的氣候，都會讓辛苦的耕耘，一夕之間變成過眼雲煙。所以農民到寺廟中祈求風調雨順，同時求籤探詢神意，就是想知道，自己的辛苦和希望會不會全都白費。以現今工商社會來看，田蠶這個項目好像已經派不上用場，不過我個人有些觀點想法，提供大家參考，田蠶這個場景雖然與現在社會情況不太符合，但是想要探詢的「神意」基礎卻是相通的，尤其是做自媒體內容創作的人，直白的說就是KOL、網紅，他們的田就是社群網路平台帳號，或是自己網站頻道，他們種得桑樹，就是自己創作出來的各種形式內容，他們細心照料養的蠶，就是點閱流量，而各位粉絲大大們就是「神意」。就像也有別的節目做籤詩的內容，但是我這樣查資料看文獻，像是做學問一樣的在介紹籤詩，就不知道各位粉絲們是不是有鍾意聽，如果我想來求神問卜看看籤詩怎麼說的話，在聖意、解籤項目，我就會看「田蠶」這個項目是怎麼解說的。",
           '預示農業或養蠶事業的運勢',
           '提示農作物或蠶絲生產',
           '暗示農業投資可能有收穫',
           '建議careful管理農業資源',
           '預示農業生產可能順利',
           '提醒關注天氣和自然條件',
           '意味著可能有農業相關的機會',
           '建議學習新的農業技術',
           '預示農業收入可能增加',
           '提示珍惜土地和自然資源',
           ],
    "尋人": ["或是說「消息」。這個部分可以看成是「尋人」，比如尋找失散的親人，行蹤不明的人。解籤的關鍵字通常會出現：「見」，見到的見、「至」，到達的至；「遲」，遲到的遲；「難」，困難的難；「杳」，音訊飄渺、杳然的杳。",
           ' 預示可能找到失散的人',
           ' 提示人際關係的重連',
           ' 暗示可能收到意外消息',
           ' 建議保持耐心和希望',
           ' 預示可能有意想不到的重逢',
           ' 提醒保持聯絡渠道暢通',
           ' 意味著可能有尋找的線索',
           ' 建議擴大搜尋範圍',
           ' 預示可能通過意外方式找到',
           ' 提示保持開放的心態',
           ],
    "行人": ["這個解籤項目有時候會跟「尋人」合併成一個。過去訊息消息傳遞不容易，第一支手機行動電話也就是1973年發明的，再加上網路的科技，才發展到今天的通訊便利，在這之前的更之前，真的是出門就像不見一樣，外地唸書的遊子、四處奔波的商人，或是常需要移動駐紮的軍人，如果杳無音訊，親人感到無助不安的時候，就會想來詢問神明，希望能從中得到這些出外的人是否平安。解籤的關鍵字一般會有：「至」、「動」、「阻」、「有信」，來提供參考。",
           '預示旅行或遷移的機會',
           '提示可能有遠距離旅行',
           '暗示旅程將順利平安',
           '建議做好旅行準備',
           '預示可能有工作或學習相關的旅行',
           '提醒注意旅途安全',
           '意味著可能有意外的旅行機會',
           '建議保持開放和靈活的心態',
           '預示旅行可能帶來新的機遇',
           '提示享受旅行的過程',
           ],
    "六甲": ["就是懷胎六甲的「六甲」，所以這個解籤項目，也稱做「子息」、「產育」。傳宗接代一向是社會價值觀裡，很重要的一環，所以來問懷孕的，或是能不能順利生產的，都在這個項目，當然也會有來問是生男還是生女的，也是看這個解籤項目。解籤的關鍵字有：「安」、「驚」、「有險」。不過，不管籤詩如何回覆您，產檢絕對還是要照時間去做。",
           '預示生育或擴大家庭的機會',
           '提示懷孕和生育的吉兆',
           '暗示家庭將迎來新生命',
           '建議做好準備迎接新成員',
           '預示母子平安',
           '提醒照顧孕婦健康',
           '意味著家庭將更加圓滿',
           '建議做好育兒準備',
           '預示子女未來可能有好運',
           '提示家庭關係將更加緊密',
           ],
    "山墳": ["又稱「風水」、「地理」、「宅墳」。風水是大家很普遍有的觀念，除了尋求堪輿老師的協助之外，一般有時候喪葬活動或是遷移墳墓的吉凶，會到寺廟求籤探詢，尤其牽涉到祖先靈寢有變化，是不是能庇蔭子孫，有時候就很可能會起家族糾紛了，所以很多時候來求這個籤，是希望神明來公正裁決。",
           '分析祖先靈位能量',
           '預測風水影響',
           '評估墓地吉凶',
           '探討家族運勢',
           '判斷祭祀效果',
           '預示家族守護力量',
           '分析風水佈局',
           '探測陰陽平衡',
           '評估祖先庇佑',
           '預測家族延續性',
           ],
    "公訟": ["也稱「訴訟」、「官司」等。不管是過去還是現在，大家都是對官司、法院這樣的地方還是敬而遠之的，但是如果實在是惹上了官司訴訟的事情，心裡面有負擔恐懼，不知道判決會如何；或是長期官司纏身，就會來請示神明。解籤的關鍵字有：「勝」、「和」、「有理」、「虧」。",
           ' 預示可能涉及法律糾紛',
           ' 提示需要理性面對衝突',
           ' 暗示法律訴訟可能有轉機',
           ' 建議尋求專業法律建議',
           ' 預示可能達成和解',
           ' 提醒保持冷靜和理性',
           ' 意味著可能需要談判',
           ' 建議蒐集充分證據',
           ' 預示訴訟可能有利解決',
           ' 提示尊重法律程序',
           ],
    "疾病": ["信眾到廟裡拜拜，一定會祈求身體健康，從個人到家人、在乎的人。在醫療不發達的年代，久病難癒實在會令人擔心、不安，所以除了向神明祈求保佑之外，也會想要求的一個解答，或是希望神明能給予一個提示，是不是病能治得好。解籤的字詞多數會使用「安」、「解」、「祈保」、「祈福」等等，或是也會有「欠安」、「難痊」這樣的字詞。不過不會出現「死」這樣的斷語。",
           '分析疾病發展趨勢',
           '預測治療效果',
           '評估身體恢復可能性',
           '探討疾病根源',
           '判斷治療方向',
           '預示醫療轉機',
           '分析身心能量',
           '探測潛在康復因素',
           '評估調理方案',
           '預測整體健康狀況',
           ],
    "失物": ["找不見的東西。在今天的時空下，不知道神明最常被問要幫忙找的，是不是寵物啊？這個解籤項目的關鍵字，會有：「見」、「難見」，也會有方位的指示，提供信眾去尋找失物。",
           '分析遺失物可尋回機率',
           '預測尋回路徑',
           '評估尋找策略',
           '探討遺失原因',
           '判斷遺失物位置',
           '預示意外發現可能',
           '分析遺失物價值',
           '探測尋找線索',
           '評估尋找時間點',
           '預測尋回結果',
           ],
    "移徙": ["也稱「遷徙」、「移居」。主要是問搬家或是搬辦公室、廠房等等。這個是看陽宅的搬遷。解籤項目多是「吉」或是「守舊」來回覆，是否合適搬遷。",
           '分析遷移可行性',
           '預測新環境適應程度',
           '評估遷移風險',
           '探討遷移動機',
           '判斷遷移時機',
           '預示遷移障礙',
           '分析新環境機會',
           '探測心理準備度',
           '評估遷移長期影響',
           '預測遷移後生活轉變',
           ],
}

DIVINE_TRAIN_DATA = defaultdict(list)
for key, value in DIVINE_LABELS.items():
    DIVINE_TRAIN_DATA[key].extend(value)

for key, value in DIVINE_Q.items():
    DIVINE_TRAIN_DATA[key].extend(value)

for key, value in DIVINE_A.items():
    DIVINE_TRAIN_DATA[key].extend(value)
print(DIVINE_TRAIN_DATA)

AUGMENTATION_Q = {
    '交易': [
        '是否能順利完成這次交易?',
        '此次買賣是否能帶來高額利潤?',
        '這筆交易是否值得投資?',
        '與對方合作是否能長期發展?',
        '交易的風險是否值得承擔?',
        '如何才能促成這次交易?',
        '與競爭對手相比，我們是否佔優勢?',
        '這次合同是否會順利簽訂?',
        '是否需要調整交易條件以達成協議?',
        '交易過程中是否會出現意外狀況?',
    ],
    '婚姻': [
        '是否能與伴侶白頭偕老?',
        '婚姻生活是否幸福美滿?',
        '如何化解婚姻中的矛盾?',
        '是否會遇到適合的婚姻對象?',
        '夫妻間的感情是否會更深厚?',
        '是否適合在今年結婚?',
        '這段感情是否會進展為婚姻?',
        '如何提升夫妻間的相處質量?',
        '是否需要外界協助解決婚姻問題?',
        '是否會有婚外感情的影響?',
    ],
    '求財': [
        '是否能在短期內獲得可觀的財富?',
        '這次投資是否會成功?',
        '是否適合創業以求財?',
        '目前的財務狀況是否會好轉?',
        '是否能得到意外之財?',
        '是否需要轉換行業來求財?',
        '如何增加收入來源?',
        '近期是否會有大筆開銷?',
        '是否適合進行高風險投資?',
        '財運是否與家人有關?',
    ],
    '自身': [
        '是否需要注意身體健康?',
        '個人的運勢是否會上升?',
        '如何提升自身的競爭力?',
        '是否適合近期做重大決策?',
        '是否會遇到影響命運的貴人?',
        '是否需要調整生活習慣以改善健康?',
        '個人的事業發展是否會順利?',
        '是否適合改變目前的生活方式?',
        '是否需要進一步學習提升技能?',
        '自身的缺點是否會影響發展?',
    ],
    '家宅': [
        '今年家運是否會旺盛?',
        '是否需要進行家宅風水調整?',
        '房屋是否會帶來吉祥運勢?',
        '家庭是否會和諧共處?',
        '是否需要搬家以提升運勢?',
        '房屋是否適合進行翻修?',
        '家庭成員是否會平安健康?',
        '是否需要進行家宅祈福?',
        '房產是否會增值?',
        '是否有外來力量影響家庭運勢?',
    ],
    '六畜': [
        '家畜是否會健康成長?',
        '是否適合擴大養殖規模?',
        '畜牧業是否會帶來穩定收益?',
        '是否需要引進新品種的家畜?',
        '家畜是否需要防疫或治療?',
        '飼養條件是否需要改善?',
        '是否適合進行跨地區交易家畜?',
        '近期是否有家畜生病的風險?',
        '畜牧業是否會受到市場波動影響?',
        '如何提升家畜的生產力?',
    ],
    '田蠶': [
        '今年的農作物是否會豐收?',
        '是否需要改良耕作技術?',
        '養蠶是否會帶來可觀的收益?',
        '是否需要考慮農業合作模式?',
        '農業活動是否會受到天氣影響?',
        '近期是否有農作物病害的風險?',
        '如何提升土地的生產力?',
        '農業收入是否能夠支撐家庭?',
        '是否適合進行農業的轉型升級?',
        '是否需要考慮擴大耕作面積?',
    ],
    '尋人': [
        '是否能夠找到失蹤的親人?',
        '尋人過程是否會遇到困難?',
        '是否需要求助專業機構尋人?',
        '尋人是否與某地區有關?',
        '尋人線索是否會出現在近期?',
        '是否能與老朋友重新聯繫?',
        '是否能找到與自己有關的貴人?',
        '失聯親人的狀況是否安全?',
        '是否適合尋找曾經的合作夥伴?',
        '尋人過程是否需要改變策略?',
    ],
    '行人': [
        '外出是否會平安順利?',
        '行人是否會按時歸來?',
        '是否需要注意旅途中的安全?',
        '是否適合近期計劃遠行?',
        '旅途中是否會遇到貴人?',
        '此次旅行是否能達成預期目的?',
        '是否需要提前取消旅行計劃?',
        '行人是否會在預期地點出現?',
        '是否適合進行商務旅行?',
        '是否需要更改旅行的目的地?',
    ],
    '六甲': [
        '是否會有懷孕的機會?',
        '孕期是否需要注意健康?',
        '是否會順利生下男孩?',
        '是否適合近期計劃生育?',
        '產檢結果是否會順利?',
        '懷孕是否會對工作造成影響?',
        '是否需要外力協助備孕?',
        '是否會有家庭成員支持孕期?',
        '新生兒是否會帶來家庭喜慶?',
        '是否需要為新生命做好準備?',
    ],
    '山墳': [
        '祖先是否安息在適合的地方?',
        '墓地風水是否會影響家運?',
        '是否需要遷移祖墳?',
        '墓地是否會受到外界干擾?',
        '是否適合近期掃墓?',
        '如何選擇吉祥的葬地?',
        '是否需要對墓地進行修繕?',
        '是否會受到祖先庇佑?',
        '墓地是否需要更改方位?',
        '是否適合進行祖先祭祀?',
    ],
    '公訟': [
        '訴訟是否會勝訴?',
        '是否需要聘請更有經驗的律師?',
        '訴訟過程是否會拖延?',
        '是否適合進行和解?',
        '此案是否會影響家庭或事業?',
        '是否需要搜集更多證據?',
        '法庭判決是否會對自己有利?',
        '訴訟費用是否會增加?',
        '是否需要尋求調解途徑?',
        '訴訟結果是否會影響未來?',
    ],
    '疾病': [
        '是否需要立即就醫?',
        '疾病是否會加重?',
        '是否能找到合適的治療方案?',
        '是否需要尋求專家意見?',
        '康復是否需要較長時間?',
        '是否適合使用某種療法?',
        '疾病是否會影響工作或生活?',
        '是否能恢復到健康狀態?',
        '是否需要注意飲食調整?',
        '是否會有復發的風險?',
    ],
    '失物': [
        '遺失的物品是否能找回?',
        '是否需要求助他人尋物?',
        '遺失物品是否與某地區有關?',
        '是否會在近期找到失物?',
        '遺失物品是否已被他人拾取?',
        '是否需要改變尋找方法?',
        '是否適合報警處理失物問題?',
        '是否會因失物影響心情或行程?',
        '是否需要放棄尋找失物?',
        '遺失的物品是否會在家中找到?',
    ],
    '移徙': [
        '是否適合近期搬家?',
        '新住處是否會帶來更好的運勢?',
        '是否需要延期遷徙計劃?',
        '搬家過程是否會遇到阻礙?',
        '新住地是否與家人需求相符?',
        '是否會因搬家影響工作或學業?',
        '移徙是否會改善家庭運勢?',
        '是否需要進一步勘察新住地?',
        '是否需要選擇特定日子搬遷?',
        '是否會因搬家影響人際關係?',
    ]
}

AUGMENTATION_DOMAINS = {
    '交易': {
        'keywords': ['買賣', '商業', '交易', '談判', '合作', '成交', '協議'],
        'synonyms': {
            '交易': ['買賣', '商業往來', '生意', '交易行為'],
            '談判': ['協商', '磋商', '商議', '討價還價'],
            '合作': ['夥伴關係', '商業聯盟', '共同發展', '戰略合作']
        }
    },
    '婚姻': {
        'keywords': ['感情', '婚姻', '戀愛', '伴侶', '姻緣', '婚配', '關係'],
        'synonyms': {
            '婚姻': ['姻緣', '婚姻關係', '夫妻生活', '婚姻狀態'],
            '感情': ['情感', '愛情', '兩性關係', '情感交流'],
            '伴侶': ['另一半', '配偶', '婚姻對象', '愛人']
        }
    },
    '求財': {
        'keywords': ['財富', '投資', '事業', '收入', '金錢', '財運', '商機'],
        'synonyms': {
            '財富': ['金錢', '財產', '收入來源', '經濟實力'],
            '投資': ['理財', '資金運用', '財務策略', '資產增值'],
            '商機': ['致富機會', '事業發展', '經濟前景', '財務契機']
        }
    },
    '自身': {
        'keywords': ['個人', '命運', '人生', '修行', '成長', '機遇', '挑戰'],
        'synonyms': {
            '命運': ['際遇', '天命', '生命軌跡', '個人宿命'],
            '修行': ['自我提升', '靈性成長', '心靈修煉', '內在修為'],
            '機遇': ['轉機', '契機', '生命轉折', '成功契機']
        }
    },
    '家宅': {
        'keywords': ['家庭', '居住', '風水', '家運', '房屋', '親情', '家族'],
        'synonyms': {
            '家庭': ['家族', '家人', '家眷', '家庭關係'],
            '風水': ['居家佈局', '居住環境', '家宅氣場', '住宅能量'],
            '家運': ['家族運勢', '家庭福祉', '家庭命運', '家族興衰']
        }
    },
    '六畜': {
        'keywords': ['家畜', '畜牧', '牛', '馬', '羊', '豬', '狗', '雞'],
        'synonyms': {
            '家畜': ['牲口', '畜生', '動物', '牧畜'],
            '牛': ['黃牛', '水牛', '公牛', '奶牛'],
            '羊': ['綿羊', '山羊', '羔羊', '羊羔']
        }
    },
    '田蠶': {
        'keywords': ['農業', '田地', '蠶桑', '耕作', '農村', '農民', '農作物'],
        'synonyms': {
            '農業': ['耕作', '種植', '田野', '農業經濟'],
            '蠶桑': ['養蠶', '蠶繭', '桑葉', '蠶絲'],
            '田地': ['農田', '耕地', '稻田', '田埂']
        }
    },
    '尋人': {
        'keywords': ['失蹤', '消息', '尋找', '線索', '離家', '下落', '失聯'],
        'synonyms': {
            '尋找': ['搜尋', '查找', '尋人啟事', '探尋'],
            '消息': ['信息', '線索', '蹤跡', '動向'],
            '失蹤': ['不見', '消失', '行蹤不明', '下落不明']
        }
    },
    '行人': {
        'keywords': ['旅行', '歸期', '出行', '路途', '旅遊', '歸來', '探親'],
        'synonyms': {
            '旅行': ['出行', '旅程', '行程', '遠行'],
            '歸期': ['回程', '返回', '歸還', '歸家'],
            '探親': ['探望親人', '回鄉探親', '親人會面', '家庭訪問']
        }
    },
    '六甲': {
        'keywords': ['生育', '子息', '懷孕', '胎兒', '子女', '親子', '孕婦'],
        'synonyms': {
            '生育': ['懷孕', '生兒育女', '生孩子', '養育'],
            '胎兒': ['嬰兒', '胎中寶寶', '小生命', '生命的萌芽'],
            '子女': ['兒女', '子孫', '孩子', '寶貝']
        }
    },
    '山墳': {
        'keywords': ['墓地', '風水', '安葬', '祖先', '山丘', '葬禮', '陵園'],
        'synonyms': {
            '墓地': ['墳地', '陵園', '公墓', '安息地'],
            '祖先': ['先人', '宗族', '祖宗', '前輩'],
            '安葬': ['葬禮', '下葬', '埋葬', '安放']
        }
    },
    '公訟': {
        'keywords': ['法律', '官司', '判決', '爭議', '糾紛', '訴訟', '法庭'],
        'synonyms': {
            '法律': ['法規', '法令', '司法', '法律制度'],
            '官司': ['訴訟', '爭訟', '訴訟案件', '法院糾紛'],
            '判決': ['裁定', '裁判', '審判結果', '法庭決定']
        }
    },
    '疾病': {
        'keywords': ['健康', '醫療', '疾病', '治療', '康復', '身體', '醫藥'],
        'synonyms': {
            '健康': ['身體狀況', '生理狀態', '體能', '身心健康'],
            '疾病': ['病痛', '醫療問題', '健康隱患', '身體不適'],
            '治療': ['醫療方案', '康復策略', '醫藥治療', '療養']
        }
    },
    '失物': {
        'keywords': ['遺失', '物品', '找回', '下落', '尋找', '線索', '丟失'],
        'synonyms': {
            '遺失': ['丟失', '失落', '失物', '不見了'],
            '物品': ['財物', '所有物', '失物', '財產'],
            '找回': ['尋回', '追回', '找回來', '失而復得']
        }
    },
    '移徙': {
        'keywords': ['搬家', '遷徙', '移動', '遷居', '住所', '新居', '離開'],
        'synonyms': {
            '搬家': ['遷居', '換地方', '移居', '住址改變'],
            '遷徙': ['移動', '搬遷', '變遷', '徙居'],
            '新居': ['新家', '新住所', '新住處', '新環境']
        }
    }
}


TEST_SENTENCES = [
    "這是一段與感情相關的提問",
    "求財運",
    "家人生病",
    "工作、升遷"
]

defaultdict(<class 'list'>, {'交易': ['交易', '買賣', '商業', '貿易', '交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。', '商業談判成功機會', '買賣交易的吉凶', '合同簽訂的風險評估', '生意夥伴的可靠性', '商業策略的正確性', '交易過程中可能的障礙', '談判中的對方態度', '價格談判的有利時機', '商業投資的潛在收益', '貿易和商業環境的變化', '跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。', '預示商業或交易的機會', '提醒交易中需要謹慎', '暗示可能有成功的商業往來', '建議仔細評估交易風險', '預示可能有重要的商業決策', '提示保持誠信和透明', '意味著交易可能需要談判', '建議做好充分的準備', '預示可能有有利的合作機會', '提醒注意合同和法律細節'], '婚姻': ['婚姻', '婚配', '結婚', '夫妻', '婚姻是兩個人結為夫妻的社會制度，通常涉及法律和情感的承諾。', '感情發展和婚姻前景', '尋找合適的婚姻對象', '婚姻是否能帶來幸福', '兩人感情的相容性', '婚姻中可能出現的問題', '伴侶的忠誠度', '婚姻生活的穩定性', '是否適合結婚', '婚姻中的磨合和成長', '感情和婚姻的命運轉折', '這是自古以來千年不變的人生大哉問！婚姻這解籤項目，當然也涵蓋了愛情問題，基本上就是親密關係的解答提示。因為人心多變、難測，只能來問神，希望能透過神明的超能力，來指引或是平復各種心情。', '預示婚姻關係的變化', '提示感情和婚姻的發展', '暗示可能遇到理想伴侶', '建議珍惜感情和溝通', '預示婚姻中的和諧與理解', '提醒處理感情中的挑戰', '意味著可能有婚姻契機', '建議保持開放和包容的態度', '預示感情生活的轉機', '提示平衡個人需求和伴侶關係'], '求財': ['求財', '財運', '財富', '資金', '求財是指追求財富和資金的行為，通常與投資和理財相關。', '投資理財的吉凶', '商業機會的成功機率', '財運起伏和金錢流向', '職業收入增長預測', '創業風險評估',

# Common Functions

## File

In [ ]:
# Protect your sensitive API_KEY, goto colab > secrets > add key
import os

from google.colab import userdata

os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')

### stopwords.txt
- Read Chinese stopwords but ignorable

In [ ]:
def load_stopwords(file_path):
    """Read Chinese stopwords."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = set(f.read().splitlines())
        print(f"Loaded {len(lines)} stopwords from {file_path}")
        return [line for line in lines if line]
    except FileNotFoundError:
        print(f"Stopwords file not found: {file_path}")
        return []


stopwords = load_stopwords('stopwords.txt') or []

Loaded 2298 stopwords from stopwords.txt


### all_divine.json
- This file is for early runs on training.
- It can also help to generate all_divine.csv later

In [ ]:
import json

all_divine_file = 'all_divine.json'
all_divine = []
# Read all_divine.json, Please upload to colab first.
with open(all_divine_file, 'r', encoding='utf8') as f:
    all_divine = json.load(f)
    print(f'Loaded {len(all_divine)} records from {all_divine_file}')

for label, texts in all_divine.items():
    print(f'{label}: {len(texts)}')
    all_divine[label] = list(set(texts))
    print(f'{label}: {len(all_divine[label])}')

Loaded 15 records from all_divine.json
交易: 209
交易: 200
婚姻: 200
婚姻: 200
求財: 200
求財: 191
自身: 200
自身: 190
家宅: 200
家宅: 191
六畜: 200
六畜: 190
田蠶: 100
田蠶: 100
尋人: 100
尋人: 100
行人: 100
行人: 100
六甲: 200
六甲: 189
山墳: 100
山墳: 100
公訟: 100
公訟: 100
疾病: 100
疾病: 100
失物: 100
失物: 100
移徙: 100
移徙: 100


### all_divine.csv
- Require all_divine.json for converting.
- Outcome is to prepare labeled_data for training after v6+.

In [ ]:
import csv

divine_csv_file = 'all_divine.csv'
with open(divine_csv_file, mode="w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["sentence", "labels"])

    for label, sentences in all_divine.items():
        for sentence in sentences:
            labels_str = ','.join(
                f"{weight[0]}:{weight[1]}" for weight in DIVINE_WEIGHTS[label])
            writer.writerow([sentence, labels_str])

print(f"Exported to CSV file {divine_csv_file}")

Exported to CSV file all_divine.csv


In [ ]:
import pandas as pd

divine_csv_file = 'all_divine.csv'
df_divine = pd.read_csv(divine_csv_file)
df_divine.head()

           sentence                              labels
0  這筆交易是否會成為公司的轉折點?  交易:1.0,求財:0.8,田蠶:0.5,六畜:0.4,行人:0.3
1   如何設計更具吸引力的交易條件?  交易:1.0,求財:0.8,田蠶:0.5,六畜:0.4,行人:0.3
2    合作協議的條款是否清晰明確?  交易:1.0,求財:0.8,田蠶:0.5,六畜:0.4,行人:0.3
3  是否需要與法律顧問再次核實合同?  交易:1.0,求財:0.8,田蠶:0.5,六畜:0.4,行人:0.3
4  是否應該制定詳細的交易後續計劃?  交易:1.0,求財:0.8,田蠶:0.5,六畜:0.4,行人:0.3


In [ ]:
import csv


def get_ids_of_labels(example_labels: list, all_labels: list) -> list:
    """Assuming self.labels contains all possible label names"""

    ids = []
    for label in example_labels:  # List of applicable labels for this input
        idx = all_labels.index(label)
        ids.append(idx)
    return ids


labeled_data = {}
with open(divine_csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        sentence = row["sentence"]
        labels = row["labels"]

        if labels not in labeled_data:
            labeled_data[labels] = []
        labeled_data[labels].append(sentence)

labels = DIVINE_LABELS.keys()
for label_str, texts in labeled_data.items():
    print()
    print('label:', label_str)
    print('sentence:', texts[0])

    tags = [kv.split(':')[0] for kv in label_str.split(',')]
    print('tag_ids:', get_ids_of_labels(
        example_labels=tags,
        all_labels=list(labels)
    ))
    break


label: 交易:1.0,求財:0.8,田蠶:0.5,六畜:0.4,行人:0.3
sentence: 這筆交易是否會成為公司的轉折點?
tag_ids: [0, 2, 6, 5, 8]


## DivineTextAugmenter

In [ ]:
import random

from transformers import BertTokenizer


class DivineTextAugmenter:
    """The class for helping sentence generation by using synonyms."""

    def __init__(self):
        # Initialize BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(
            "ckiplab/bert-base-chinese")
        self.augmentation_domains = AUGMENTATION_DOMAINS

    def augment_text(
        self,
        domain: str,
        original_text: str,
        num_of_new_texts=3,
        augmentation_rate=0.3,
    ) -> list:
        if domain not in self.augmentation_domains:
            return [original_text]

        augmented_texts = [original_text]
        tokens = self.tokenizer.tokenize(original_text)

        for _ in range(num_of_new_texts):
            if random.random() < augmentation_rate:
                augmented_tokens = tokens.copy()
                domain_keywords = self.augmentation_domains[domain]['keywords']
                domain_synonyms = self.augmentation_domains[domain]['synonyms']

                for i, token in enumerate(augmented_tokens):
                    for keyword, synonyms in domain_synonyms.items():
                        if token == keyword:
                            augmented_tokens[i] = random.choice(synonyms)

                if random.random() < 0.5:
                    contextual_phrases = [
                        f"關於{random.choice(domain_keywords)}的困擾",
                        f"希望解決{random.choice(domain_keywords)}的問題",
                        f"詢問{random.choice(domain_keywords)}的未來"
                    ]
                    contextual_tokens = self.tokenizer.tokenize(
                        random.choice(contextual_phrases))
                    augmented_tokens.extend(contextual_tokens)

                augmented_text = self.tokenizer.convert_tokens_to_string(
                    augmented_tokens)
                augmented_texts.append(augmented_text)

        return list(set(augmented_texts))

##BertForMultiLabelClassification

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel


class BertForMultiLabelClassification(BertPreTrainedModel):
    """The class with a custom model that supports multi-label classification."""

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.sigmoid = nn.Sigmoid()  # Sigmoid for multi-label output

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True
        )
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        probabilities = self.sigmoid(logits)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
            loss = loss_fct(logits, labels.float())  # Multi-hot encoded labels

        return {"loss": loss, "logits": probabilities}

##Callbacks
- Pluggable callbacks for training.
- This project doesn't enable early_stop, instead, would like to monitor from end to end.

In [ ]:
import logging

from transformers import EarlyStoppingCallback, TrainerCallback

logging.basicConfig(level=logging.INFO)


class CustomCallback(TrainerCallback):
    def on_step_end(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            print(f"Step: {state.global_step}, Logs: {logs}")


early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=10,  # Stop if no improvement for 10 epochs
    early_stopping_threshold=0.01  # Minimum change to qualify as improvement
)

## BERTTrainer
- The trainer used in the early training runs (v1~v5)
- This is for label prediction.

```py
# Usage example:
labels = DIVINE_LABELS.keys()
trainer = BERTTrainer("ckiplab/bert-base-chinese", list(labels), all_divine, stopwords=stopwords)
trainer.train(training_args=training_args)
trainer.save_model()
```

In [ ]:
from datasets import Dataset
from transformers import BertForSequenceClassification, BertTokenizer, Trainer


class BERTTrainer:
    """The trainer for label prediction."""

    def __init__(self,
                 model_name: str,
                 labels: list,
                 divine_data: dict,
                 stopwords: list = None,
                 ):
        self.model_name = model_name
        self.labels = list(labels)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=len(labels)  # 15 labels
        )

        self.label_to_id = {label: idx for idx, label in enumerate(labels)}
        self.id_to_label = {idx: label for label,
                            idx in self.label_to_id.items()}
        self.stopwords = stopwords or []
        self.divine_data = divine_data.copy()
        self.dataset = Dataset.from_dict({
            "text": ['\n'.join(docs) for docs in divine_data.values()],
            "label": [self.label_to_id[label] for label in labels]
        })

    def preprocess_function(self):
        def tokenize_and_filter(examples):
            tokens = self.tokenizer(
                examples["text"], padding="max_length", truncation=True)
            new_tokens = []

            for token_list in tokens['input_ids']:
                filtered_tokens = [token for token in token_list if self.tokenizer.decode(
                    token) not in self.stopwords]
                padded_tokens = filtered_tokens[:
                                                self.tokenizer.model_max_length]
                padded_tokens += [self.tokenizer.pad_token_id] * \
                    (self.tokenizer.model_max_length - len(padded_tokens))
                new_tokens.append(padded_tokens)

            tokens['input_ids'] = new_tokens
            return tokens
        return self.dataset.map(tokenize_and_filter, batched=True)

    def run_augmentation(self, num_of_new_texts: int = 3) -> dict:
        self.augmenter = DivineTextAugmenter()
        divine_data = {}
        for label, texts in self.divine_data.items():
            augmented_texts: list = []
            for text in texts:
                new_data = self.augmenter.augment_text(
                    domain=label,
                    original_text=text,
                    num_of_new_texts=num_of_new_texts,
                )
                new_lines = [''.join(text.split())
                             for i, text in enumerate(new_data, 1)]
                print(f'Augmented texts:', new_lines)
                augmented_texts.extend(new_lines)
            divine_data[label] = list(set(augmented_texts))

        # overwrite
        self.dataset = Dataset.from_dict({
            "text": ['\n'.join(docs) for docs in divine_data.values()],
            "label": [self.label_to_id[label] for label in labels]
        })
        return divine_data

    def split_dataset(self, source):
        print("Splitting dataset...")
        train_dataset = source.shuffle(seed=42).select(
            range(int(0.8 * len(source))))
        eval_dataset = source.shuffle(seed=42).select(
            range(int(0.8 * len(source)), len(source)))
        return train_dataset, eval_dataset

    def train(self, training_args, callbacks: list = None):
        print("Training BERT model...")
        tokenized_dataset = self.preprocess_function()

        train_dataset, eval_dataset = self.split_dataset(tokenized_dataset)
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            callbacks=callbacks,
        )
        # trainer.add_callback(early_stopping_callback)
        # trainer.add_callback(CustomCallback())

        trainer.train()
        print("Training completed.")
        return trainer

    def save_model(self, path="./fine-tuned-bert"):
        print(f"Saving fine-tuned BERT model to {path}...")
        self.model.save_pretrained(path)
        print("Fine-tuned BERT model saved to", path)
        self.tokenizer.save_pretrained(path)
        print("Tokenizer saved to", path)

# Usage example:
# labels = DIVINE_LABELS.keys()
# trainer = BERTTrainer("ckiplab/bert-base-chinese", list(labels), all_divine, stopwords=stopwords)
# trainer.train(training_args=training_args)
# trainer.save_model()

## BERTMultiLabelTrainer
- The trainer used in the recent training runs (v6+)
- This is for cross classification, aimed to predict multiple labels.

```py
# Usage example:
labels = DIVINE_LABELS.keys()
trainer = BERTMultiLabelTrainer("ckiplab/bert-base-chinese", list(labels), labeled_data, stopwords=stopwords)
trainer.train(training_args=training_args)
trainer.save_model()
```

In [ ]:
import numpy as np
from datasets import Dataset
from transformers import BertTokenizer, EarlyStoppingCallback, Trainer


class BERTMultiLabelTrainer:
    """The trainer for cross classification, aimed to predict multiple labels."""

    def __init__(self,
                 model_name: str,
                 labels: list,
                 divine_data: dict,
                 stopwords: list = None,
                 ):
        self.model_name = model_name
        self.labels = list(labels)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        # Initialize model with custom multi-label class
        self.model = BertForMultiLabelClassification.from_pretrained(
            model_name,
            num_labels=len(self.labels)  # 15 labels
        )

        self.label_to_id = {label: idx for idx, label in enumerate(labels)}
        self.id_to_label = {idx: label for label,
                            idx in self.label_to_id.items()}
        self.stopwords = stopwords or []
        self.divine_data = divine_data.copy()

        # Update dataset creation
        self.dataset = Dataset.from_dict({
            "text": ['\n'.join(docs) for docs in divine_data.values()],
            "label": [self.encode_labels(
                [kv.split(':')[0] for kv in label_str.split(',')], self.labels)
                for label_str in divine_data.keys()]
        })

    def encode_labels(self, example_labels: list, all_labels: list) -> list:
        """Assuming self.labels contains all possible label names"""
        multi_hot = np.zeros(len(all_labels), dtype=int)
        for label in example_labels:
            idx = all_labels.index(label)
            multi_hot[idx] = 1
        return multi_hot

    def preprocess_function(self):
        def tokenize_and_filter(examples):
            tokens = self.tokenizer(
                examples["text"], padding="max_length", truncation=True)
            new_tokens = []

            for token_list in tokens['input_ids']:
                filtered_tokens = [token for token in token_list if self.tokenizer.decode(
                    token) not in self.stopwords]
                padded_tokens = filtered_tokens[:
                                                self.tokenizer.model_max_length]
                padded_tokens += [self.tokenizer.pad_token_id] * \
                    (self.tokenizer.model_max_length - len(padded_tokens))
                new_tokens.append(padded_tokens)

            tokens['input_ids'] = new_tokens
            return tokens
        return self.dataset.map(tokenize_and_filter, batched=True)

    def run_augmentation(self, num_of_new_texts: int = 3) -> dict:
        self.augmenter = DivineTextAugmenter()
        divine_data = {}
        for label, texts in self.divine_data.items():
            augmented_texts: list = []
            for text in texts:
                new_data = self.augmenter.augment_text(
                    domain=label,
                    original_text=text,
                    num_of_new_texts=num_of_new_texts,
                )
                new_lines = [''.join(text.split())
                             for i, text in enumerate(new_data, 1)]
                print(f'Augmented texts:', new_lines)
                augmented_texts.extend(new_lines)
            divine_data[label] = list(set(augmented_texts))

        # overwrite
        self.dataset = Dataset.from_dict({
            "text": ['\n'.join(docs) for docs in divine_data.values()],
            "label": [self.encode_labels(
                [kv.split(':')[0] for kv in label_str.split(',')], self.labels)
                for label_str in divine_data.keys()]
        })
        return divine_data

    def split_dataset(self, source):
        print("Splitting dataset...")
        train_dataset = source.shuffle(seed=42).select(
            range(int(0.8 * len(source))))
        eval_dataset = source.shuffle(seed=42).select(
            range(int(0.8 * len(source)), len(source)))
        return train_dataset, eval_dataset

    def train(self, training_args, callbacks: list = None):
        print("Training BERT model...")
        tokenized_dataset = self.preprocess_function()

        train_dataset, eval_dataset = self.split_dataset(tokenized_dataset)
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            callbacks=callbacks,
        )
        # trainer.add_callback(early_stopping_callback),
        # trainer.add_callback(CustomCallback())

        trainer.train()
        print("Training completed.")
        return trainer

    def save_model(self, path="./fine-tuned-bert"):
        print(f"Saving fine-tuned BERT model to {path}...")
        self.model.save_pretrained(path)
        print("Fine-tuned BERT model saved to", path)
        self.tokenizer.save_pretrained(path)
        print("Tokenizer saved to", path)

## Embedding

In [ ]:
from typing import Callable

import torch
from sklearn.metrics.pairwise import cosine_similarity


def get_word_embedding_by_model(model: Callable, word: str) -> torch.Tensor:
    """Generate the embedding of a word using a sentence transformer model."""
    embedding = model.encode(word, convert_to_tensor=True)
    return embedding.cpu() if embedding.device.type == 'cuda' else embedding


def compute_cosine_similarity_by_model(model: Callable, word1: str, word2: str) -> float:
    """Compute the cosine similarity between two words using a sentence transformer model."""
    embedding1 = get_word_embedding_by_model(model, word1)
    embedding2 = get_word_embedding_by_model(model, word2)
    similarity = cosine_similarity(embedding1.numpy().reshape(
        1, -1), embedding2.numpy().reshape(1, -1))
    return similarity[0][0]


def get_word_embedding(tokenizer, model, word: str):
    """Generate the embedding of a word using a sentence transformer model."""
    tokens = tokenizer(word, return_tensors="pt", add_special_tokens=True)
    with torch.no_grad():
        outputs = model(**tokens)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.cpu() if cls_embedding.device.type == 'cuda' else cls_embedding


def compute_cosine_similarity(tokenizer, model, word1: str, word2: str):
    """Compute the cosine similarity between two words using a sentence transformer model."""
    embedding1 = get_word_embedding(tokenizer, model, word1)
    embedding2 = get_word_embedding(tokenizer, model, word2)
    similarity = cosine_similarity(embedding1.numpy(), embedding2.numpy())
    return similarity[0][0]

## Train

- Unorganized functions, testing purposing

In [ ]:
from typing import Any, Callable, Optional

from datasets import Dataset


def tokenize_function(tokenizer: Callable, dataset: Dataset) -> Dataset:
    "Tokenize the dataset with padding and truncation."
    return tokenizer(dataset["text"], padding="max_length", truncation=True)


def preprocess_function(
    tokenizer: Callable,
    dataset: Dataset,
    stopwords: Optional[list[str]] = None
) -> Dataset:
    "Preprocess the dataset by tokenizing and filtering stopwords."

    def tokenize_and_filter(examples: dict[str, Any]) -> dict[str, Any]:
        tokens = tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True
        )
        new_tokens = []
        for token_list in tokens['input_ids']:
            filtered_tokens = [token for token in token_list if tokenizer.decode(
                token) not in (stopwords or [])]
            padded_tokens = filtered_tokens[:tokenizer.model_max_length]
            padded_tokens += [tokenizer.pad_token_id] * \
                (tokenizer.model_max_length - len(padded_tokens))
            new_tokens.append(padded_tokens)

        tokens['input_ids'] = new_tokens
        return tokens

    tokenized_dataset = dataset.map(tokenize_and_filter, batched=True)
    return tokenized_dataset

## Predict

- Threshold
```py
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
logits = outputs.logits
predictions = logits > threshold
predicted_ids_with_scores = [
    (idx, logits[0][idx].item())
    for idx, pred in enumerate(predictions[0])
    if pred
]
return predicted_ids_with_scores
```    

- TopN labels (and scores)
```py
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
logits = outputs.logits
probs = torch.nn.functional.softmax(logits, dim=-1)
topk_values, topk_indices = torch.topk(probs, top_k)
return [
    (idx.item(), prob.item())
    for idx, prob in zip(topk_indices[0], topk_values[0])
    if prob.item() >= threshold
]
```    

In [ ]:
# Utilities for prediction and similarity evaluation
import torch


def predict(tokenizer, model, text: str) -> int:
    """Predict the top-1 label for the given text."""

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    return logits.argmax().item()


def predict_top_k(
    tokenizer, model, text: str, top_k: int = 3
) -> list[tuple[int, float]]:
    """Predict the top-k labels and their probabilities for the given text."""

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    topk_values, topk_indices = torch.topk(probs, top_k)
    return [
        (idx.item(), prob.item()) for idx, prob in zip(topk_indices[0], topk_values[0])
    ]


def predict_threshold(tokenizer, model, text: str, threshold: float = 0.5) -> list[int]:
    """Predict labels with probabilities above the given threshold."""

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits > threshold
    return [idx for idx, pred in enumerate(predictions[0]) if pred]


def predict_top_k_threshold(
    tokenizer, model, text: str, top_k: int = 3, threshold: float = 0.5
) -> list[tuple[int, float]]:
    """Predict the top-k labels with their probabilities, filtered by a threshold."""

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    topk_values, topk_indices = torch.topk(probs, top_k)
    return [
        (idx.item(), prob.item())
        for idx, prob in zip(topk_indices[0], topk_values[0])
        if prob.item() >= threshold
    ]


def test_model_similarity_on_divine(model):
    """Evaluate similarity of model predictions with predefined divine labels."""

    similarity_pairs = {text: [] for text in TEST_SENTENCES}
    for text in TEST_SENTENCES:
        for label in DIVINE_LABELS.keys():
            similarity = compute_cosine_similarity_by_model(model, text, label)
            similarity_pairs[text].append((label, similarity))

    for text, similarity_list in similarity_pairs.items():
        print(f"{text}:{'='*10}")
        top_labels = sorted(similarity_list, key=lambda x: x[1], reverse=True)[:3]
        for label, score in top_labels:
            print(f"Label: {label}, Score: {score:.4f}")


# Augmentation Tests

## DivineTextAugmenter

In [ ]:
# Testing
test_augmenter = DivineTextAugmenter()
augmented_data = {}

for label, texts in DIVINE_TRAIN_DATA.items():
    augmented_texts = []
    for text in texts:
        augmented_texts += (
            test_augmenter.augment_text(
                domain=label,
                original_text=text,
                num_of_new_texts=3,
            )
            or []
        )

    augmented_data[label] = augmented_texts
    print(f"{label}:", "=" * 20)
    print(
        "\n".join(
            [
                f"{i}:" + "".join(text.split())
                for i, text in enumerate(augmented_texts, 1)
            ]
        )
    )

    break

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

交易: ====================
1:交易
2:買賣
3:買賣
4:商業希望解決商業的問題
5:商業詢問協議的未來
6:商業
7:貿易詢問合作的未來
8:貿易
9:貿易關於合作的困擾
10:交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。希望解決商業的問題
11:交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。
12:交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。
13:商業談判成功機會
14:商業談判成功機會
15:買賣交易的吉凶
16:合同簽訂的風險評估希望解決談判的問題
17:合同簽訂的風險評估
18:生意夥伴的可靠性
19:生意夥伴的可靠性希望解決合作的問題
20:商業策略的正確性
21:交易過程中可能的障礙希望解決商業的問題
22:交易過程中可能的障礙
23:談判中的對方態度
24:價格談判的有利時機
25:商業投資的潛在收益關於成交的困擾
26:商業投資的潛在收益
27:貿易和商業環境的變化希望解決談判的問題
28:貿易和商業環境的變化
29:貿易和商業環境的變化
30:跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。
31:跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。關於協議的困擾
32:預示商業或交易的機會
33:預示商業或交易的機會
34:提醒交易中需要謹慎
35:暗示可能有成功的商業往來
36:暗示可能有成功的商業往來希望解決交易的問題
37:建議仔細評估交易風險
38:預示可能有重要的商業決策
39:預示可能有重要的商業決策
40:提示保持誠信和透明
41:提示保持誠信和透明詢問商業的未來
42:提示保持誠信和透明詢問談判的未來
43:提示保持誠信和透明
44:意味著交易可能需要談判
45:建議做好充分的準備
46:預示可能有有利的合作機會
47:提醒注意合同和法律細節


In [ ]:
print(DIVINE_TRAIN_DATA)
labels = DIVINE_LABELS.keys()
augmenter = DivineTextAugmenter()
divine_data = DIVINE_TRAIN_DATA.copy()

for label, texts in divine_data.items():
    print(f"Augmenting texts for label: {label}")
    augmented_texts: list = []
    for text in texts:
        print(f"Original text: {text}")
        new_data = augmenter.augment_text(
            domain=label,
            original_text=text,
            num_of_new_texts=3,
        )
        new_lines = [''.join(text.split())
                     for i, text in enumerate(new_data, 1)]
        augmented_texts.extend(new_lines)
    divine_data[label] = augmented_texts.copy()

divine_data

defaultdict(<class 'list'>, {'交易': ['交易', '買賣', '商業', '貿易', '交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。', '商業談判成功機會', '買賣交易的吉凶', '合同簽訂的風險評估', '生意夥伴的可靠性', '商業策略的正確性', '交易過程中可能的障礙', '談判中的對方態度', '價格談判的有利時機', '商業投資的潛在收益', '貿易和商業環境的變化', '跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。', '預示商業或交易的機會', '提醒交易中需要謹慎', '暗示可能有成功的商業往來', '建議仔細評估交易風險', '預示可能有重要的商業決策', '提示保持誠信和透明', '意味著交易可能需要談判', '建議做好充分的準備', '預示可能有有利的合作機會', '提醒注意合同和法律細節'], '婚姻': ['婚姻', '婚配', '結婚', '夫妻', '婚姻是兩個人結為夫妻的社會制度，通常涉及法律和情感的承諾。', '感情發展和婚姻前景', '尋找合適的婚姻對象', '婚姻是否能帶來幸福', '兩人感情的相容性', '婚姻中可能出現的問題', '伴侶的忠誠度', '婚姻生活的穩定性', '是否適合結婚', '婚姻中的磨合和成長', '感情和婚姻的命運轉折', '這是自古以來千年不變的人生大哉問！婚姻這解籤項目，當然也涵蓋了愛情問題，基本上就是親密關係的解答提示。因為人心多變、難測，只能來問神，希望能透過神明的超能力，來指引或是平復各種心情。', '預示婚姻關係的變化', '提示感情和婚姻的發展', '暗示可能遇到理想伴侶', '建議珍惜感情和溝通', '預示婚姻中的和諧與理解', '提醒處理感情中的挑戰', '意味著可能有婚姻契機', '建議保持開放和包容的態度', '預示感情生活的轉機', '提示平衡個人需求和伴侶關係'], '求財': ['求財', '財運', '財富', '資金', '求財是指追求財富和資金的行為，通常與投資和理財相關。', '投資理財的吉凶', '商業機會的成功機率', '財運起伏和金錢流向', '職業收入增長預測', '創業風險評估',

defaultdict(list,
            {'交易': ['交易',
              '買賣關於談判的困擾',
              '買賣',
              '商業',
              '貿易',
              '貿易關於買賣的困擾',
              '交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。',
              '商業談判成功機會詢問談判的未來',
              '商業談判成功機會',
              '買賣交易的吉凶關於談判的困擾',
              '買賣交易的吉凶詢問成交的未來',
              '買賣交易的吉凶',
              '合同簽訂的風險評估',
              '合同簽訂的風險評估',
              '生意夥伴的可靠性',
              '生意夥伴的可靠性關於成交的困擾',
              '商業策略的正確性關於交易的困擾',
              '商業策略的正確性詢問交易的未來',
              '商業策略的正確性',
              '交易過程中可能的障礙詢問談判的未來',
              '交易過程中可能的障礙關於交易的困擾',
              '交易過程中可能的障礙',
              '談判中的對方態度希望解決商業的問題',
              '談判中的對方態度',
              '價格談判的有利時機',
              '商業投資的潛在收益',
              '商業投資的潛在收益',
              '貿易和商業環境的變化',
              '貿易和商業環境的變化關於買賣的困擾',
              '跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。',
              '跟地緣有關，艋舺龍山寺過去是商業重地，廟方

## Translation

In [ ]:
from googletrans import Translator


def translate_text(text: str, source_lang: str, target_lang: str) -> str:
    "Translates text between Chinese Traditional and English using Google Translate."

    translator = Translator()
    translation = translator.translate(text, src=source_lang, dest=target_lang)
    return translation.text


# Example usage:
chinese_text = "這是中文句子。"
english_text = "This is an English sentence."

# Translate Chinese to English
english_translation = translate_text(chinese_text, 'zh-tw', 'en')
print(f"Chinese: {chinese_text}")
print(f"English Translation: {english_translation}")

# Translate English to Chinese
chinese_translation = translate_text(english_text, 'en', 'zh-tw')
print(f"English: {english_text}")
print(f"Chinese Translation: {chinese_translation}")

for label, texts in AUGMENTATION_Q.items():
    print(f"{label}:", '='*20)
    for text in texts:
        print(f"Original text: {text}")
        translated_text = translate_text(text, 'zh-tw', 'en')
        print(f"Translated text: {translated_text}")
    break

Chinese: 這是中文句子。
English Translation: This is a Chinese sentence.
English: This is an English sentence.
Chinese Translation: 這是英語句子。
交易: ====================
Original text: 是否能順利完成這次交易?
Translated text: Can I successfully complete this transaction?
Original text: 此次買賣是否能帶來高額利潤?
Translated text: Can this sale bring high profits?
Original text: 這筆交易是否值得投資?
Translated text: Is this transaction worth investing?
Original text: 與對方合作是否能長期發展?
Translated text: Can cooperation with the other party develop for a long time?
Original text: 交易的風險是否值得承擔?
Translated text: Is the risk of transactions worthwhile?
Original text: 如何才能促成這次交易?
Translated text: How can we promote this transaction?
Original text: 與競爭對手相比，我們是否佔優勢?
Translated text: Compared with competitors, do we have an advantage?
Original text: 這次合同是否會順利簽訂?
Translated text: Will this contract be signed smoothly?
Original text: 是否需要調整交易條件以達成協議?
Translated text: Do I need to adjust the trading conditions to reach an agreement?
Original text: 

## SynonymAug

In [ ]:
import random

import nlpaug.augmenter.word as naw
import nltk
from googletrans import Translator

nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('wordnet')
# nltk.download('omw-1.4')


def translate_and_augment_zh_tw(
    text: str,
    num_aug: int = 3,
    aug_p: float = 0.2,
    aug_min: int = 1,
    aug_max: int = 2
) -> list[str]:
    "Translates Chinese Traditional text to English and augments the English text."
    try:
        # Translate Chinese to English using Google Translate
        translator = Translator()
        english_text = translator.translate(text, src='zh-tw', dest='en').text

        # Initialize SynonymAug with conservative settings
        aug = naw.SynonymAug(aug_src='wordnet', aug_p=aug_p,
                             aug_min=aug_min, aug_max=aug_max)

        # Generate augmented English sentences
        augmented_sentences = [aug.augment(
            english_text) for _ in range(num_aug)]
        return augmented_sentences

    except Exception as e:
        print(f"Error during translation or augmentation: {e}")
        return [text]  # Return original text if any error occurs


# Example usage
chinese_text = "商業談判成功機會"
aug_p = 1.0 if len(chinese_text) > 20 else 2.0
augmented_english_sentences = translate_and_augment_zh_tw(
    text=chinese_text,
    num_aug=3,
    aug_p=aug_p + 0.8 * random.random(),
    aug_min=max(1, len(chinese_text) // 2),
    aug_max=max(2, len(chinese_text) * 2)
)

for sentence in augmented_english_sentences:
    print(sentence)
    if isinstance(sentence, str):
        print(translate_text(sentence, 'en', 'zh-tw'))
    elif isinstance(sentence, list):
        print(translate_text('\n'.join(sentence), 'en', 'zh-tw'))
    else:
        print(f"Skipping translation for non-string: {sentence}")

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


['Opportunity for successful business negotiations']
成功商業談判的機會
['Chance for successful business negotiations']
成功商業談判的機會
['Opportunities for successful line negotiations']
成功談判的機會


## gpt2

In [ ]:
from typing import Callable

from transformers import BertTokenizer, GPT2LMHeadModel, GPT2Tokenizer

gp2_model = GPT2LMHeadModel.from_pretrained("ckiplab/gpt2-base-chinese")
# gp2_tokenizer = GPT2Tokenizer.from_pretrained("ckiplab/gpt2-base-chinese")
gp2_tokenizer = BertTokenizer.from_pretrained("ckiplab/bert-base-chinese")


def generate_similar_sentence(
    tokenizer: Callable,
    model: Callable,
    text: str,
    max_length: int = 50,
    temperature: float = 1.0,
    top_k: int = 50,
    top_p: float = 0.9
) -> str:
    "Generate a similar sentence based on input text."
    inputs = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def augment_textlist(
    tokenizer: Callable,
    model: Callable,
    data: list[str],
    max_length: int = 50,
    temperature: float = 1.0,
    top_k: int = 50,
    top_p: float = 0.9
) -> list[str]:
    "Augment a list of text with similar sentences."
    return [
        generate_similar_sentence(
            tokenizer=tokenizer,
            model=model,
            text=text,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p
        )
        for text in data
    ]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/421M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

In [ ]:
# Testing
augmented_data = {}
for label, texts in DIVINE_LABELS.items():
    augmented_texts = augment_textlist(
        tokenizer=gp2_tokenizer,
        model=gp2_model,
        data=texts,
        max_length=50,
        temperature=1.0,
        top_k=50,
        top_p=0.9
    )
    augmented_data[label] = augmented_texts
    print(f"{label}:", '='*20)
    print('\n'.join([f'{i}:' + ''.join(text.split())
                     for i, text in enumerate(augmented_texts, 1)]))
    break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


交易: ====================
1:交易數上漲五點七三,以二四四十點八六收盤,漲幅百分之一點零五。今天的成交量較前一個交易日的三億四
2:買賣場投資人對股票的意願,因此股票的投資應該和美國證券交易所一致。」900629南投縣政府為使台中縣政
3:商業行(of)今天報導,今年上半年香港銀行的存款利率仍然疲弱。渣打銀行()上半年經常帳
4:貿易發展協定的內涵,這個協定的內涵是以「貿易總協定」和「貿聯標準」為基礎,並由大陸「國家發展計劃
5:交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。交易的特性，從交易性的內涵，以提高交易意見。


In [ ]:
# Testing data augmentation
augmented_data = {}
for label, texts in DIVINE_LABELS.items():
    augmented_texts = augment_textlist(
        tokenizer=gp2_tokenizer,
        model=gp2_model,
        data=texts,
        max_length=50,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    augmented_data[label] = augmented_texts
    print(f"{label}:", '='*20)
    print('\n'.join([f'{i}:' + ''.join(text.split())
                     for i, text in enumerate(augmented_texts, 1)]))
    break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


交易: ====================
1:交易認為,美國國際油價可能上漲,加上國內油價持續走低,台北市短期內油價應該會維持在每桶50美元左右
2:買賣不同的價格都有差別,但買賣價格可以一次下降,以往有些人以為買賣價格會上升,而是要求買賣價格在
3:商業動。中華民國經濟部長王志剛今天說,中共在東南亞的「重大成就」,對東南亞國家來說,是最好的例子
4:貿易表署說,將為中國大陸改善關係提供一個更好的條件。這份聲明說:「關貿總協的一個目標是在推動兩個
5:交易是指買賣商品或服務的行為，涉及金錢和貨物的交換。國大陸，金融機構和個人的財產可以依法向金融機


# Use pre-trained model

## BertClassification - ckiplab/bert-base-chinese

In [ ]:
from transformers import BertModel, BertTokenizer

pre_tokenizer = BertTokenizer.from_pretrained('ckiplab/bert-base-chinese')
pre_model = BertModel.from_pretrained('ckiplab/bert-base-chinese')

Some weights of BertModel were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
similarity_pairs = {text: [] for text in TEST_SENTENCES}
for text in TEST_SENTENCES:
    for label in DIVINE_LABELS.keys():
        word1 = text
        word2 = label
        similarity = compute_cosine_similarity(
            pre_tokenizer, pre_model, word1, word2)
        similarity_pairs[text].append((label, similarity))

for text, similarity_list in similarity_pairs.items():
    print(f"{text}:{'='*10}")
    top_labels = sorted(similarity_list, key=lambda x: x[1], reverse=True)[:3]
    for label, score in top_labels:
        print(f"Label: {label}, Score: {score:.4f}")

這是一段與感情相關的提問:==========
Label: 自身, Score: 0.7377
Label: 交易, Score: 0.7210
Label: 六甲, Score: 0.7134
求財運:==========
Label: 求財, Score: 0.9002
Label: 家宅, Score: 0.7499
Label: 田蠶, Score: 0.7413
家人生病:==========
Label: 疾病, Score: 0.8383
Label: 自身, Score: 0.7947
Label: 行人, Score: 0.7845
工作、升遷:==========
Label: 自身, Score: 0.8424
Label: 六甲, Score: 0.8408
Label: 田蠶, Score: 0.8408


## SentenceTransformer - paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
from sentence_transformers import SentenceTransformer

paraphrase_model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
test_model_similarity_on_divine(paraphrase_model)

這是一段與感情相關的提問:==========
Label: 尋人, Score: 0.4725
Label: 自身, Score: 0.3932
Label: 求財, Score: 0.3293
求財運:==========
Label: 求財, Score: 0.9539
Label: 交易, Score: 0.7383
Label: 尋人, Score: 0.6301
家人生病:==========
Label: 疾病, Score: 0.5064
Label: 失物, Score: 0.3659
Label: 公訟, Score: 0.2886
工作、升遷:==========
Label: 移徙, Score: 0.4977
Label: 求財, Score: 0.4932
Label: 行人, Score: 0.4726


## SentenceTransformer - distiluse-base-multilingual-cased-v2

In [ ]:
from sentence_transformers import SentenceTransformer

distiluse_model = SentenceTransformer(
    'sentence-transformers/distiluse-base-multilingual-cased-v2')

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
test_model_similarity_on_divine(distiluse_model)

這是一段與感情相關的提問:==========
Label: 婚姻, Score: 0.0780
Label: 尋人, Score: 0.0650
Label: 求財, Score: 0.0511
求財運:==========
Label: 求財, Score: 0.8898
Label: 尋人, Score: 0.6541
Label: 失物, Score: 0.5772
家人生病:==========
Label: 疾病, Score: 0.6435
Label: 家宅, Score: 0.4307
Label: 婚姻, Score: 0.4110
工作、升遷:==========
Label: 行人, Score: 0.4566
Label: 尋人, Score: 0.4455
Label: 移徙, Score: 0.3838


# Fine-tune model

## v1 ckiplab/bert-base-chinese

Inital try

In [ ]:
from datasets import Dataset
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments

v1_train_tokenizer = BertTokenizer.from_pretrained("ckiplab/bert-base-chinese")
v1_train_model = BertForSequenceClassification.from_pretrained(
    "ckiplab/bert-base-chinese",
    num_labels=len(DIVINE_LABELS.keys()),  # 15 labels
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Double check expected label and id mapping

test_label_to_id = {label: idx for idx,
                    label in enumerate(DIVINE_LABELS.keys())}
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}

test_train_dataset = Dataset.from_dict({
    "text": ['\n'.join(docs) for docs in DIVINE_Q.values()],
    "label": [test_label_to_id[label] for label in DIVINE_LABELS.keys()]
})

v1_train_tokenized_datasets = preprocess_function(
    tokenizer=v1_train_tokenizer,
    dataset=test_train_dataset,
    stopwords=stopwords or []
)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

v1_training_args = TrainingArguments(
    run_name="v1",
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=1,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
print(v1_train_tokenized_datasets[0])  # Check the first sample

{'text': '跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。', 'label': 0, 'input_ids': [101, 5225, 7302, 5674, 5671, 7983, 2255, 2191, 1555, 3511, 7028, 2448, 3175, 6880, 4245, 4415, 2382, 1555, 4518, 1894, 2448, 6174, 1301, 1558, 769, 3211, 752, 2658, 2746, 1281, 1818, 4495, 3833, 1147, 6716, 7028, 7517, 4680, 2961, 6210, 928, 4707, 7028, 6213, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
v1_trainer = Trainer(
    model=v1_train_model,
    args=v1_training_args,
    train_dataset=v1_train_tokenized_datasets,
    eval_dataset=v1_train_tokenized_datasets,
)
v1_trainer.add_callback(CustomCallback())

# OOM in CPU mode
v1_trainer.train()

# cat ./logs/trainer_state.json

Epoch,Training Loss,Validation Loss
1,No log,2.628546
2,No log,2.563735
3,No log,2.498255
4,No log,2.397807
5,No log,2.381611
6,No log,2.325981
7,No log,2.275013
8,No log,2.202114
9,No log,2.095918
10,2.453400,2.002733


TrainOutput(global_step=100, training_loss=0.720313025712967, metrics={'train_runtime': 192.4707, 'train_samples_per_second': 7.793, 'train_steps_per_second': 0.52, 'total_flos': 394712649216000.0, 'train_loss': 0.720313025712967, 'epoch': 100.0})

In [ ]:
v1_train_model.save_pretrained("./fine-tuned-bert")
v1_train_tokenizer.save_pretrained("./fine-tuned-bert")

('./fine-tuned-bert/tokenizer_config.json',
 './fine-tuned-bert/special_tokens_map.json',
 './fine-tuned-bert/vocab.txt',
 './fine-tuned-bert/added_tokens.json')

### Predict - BertClassification

In [ ]:
# Double check expected label and id mapping

test_label_to_id = {label: idx for idx,
                    label in enumerate(DIVINE_LABELS.keys())}
print('test label_to_id:', test_label_to_id)
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}
print('test id_to_label:', test_id_to_label)
print('test text[0]:', ['\n'.join(docs) for docs in DIVINE_Q.values()][0])
print('test labels:', [test_label_to_id[label]
      for label in DIVINE_LABELS.keys()])

test label_to_id: {'交易': 0, '婚姻': 1, '求財': 2, '自身': 3, '家宅': 4, '六畜': 5, '田蠶': 6, '尋人': 7, '行人': 8, '六甲': 9, '山墳': 10, '公訟': 11, '疾病': 12, '失物': 13, '移徙': 14}
test id_to_label: {0: '交易', 1: '婚姻', 2: '求財', 3: '自身', 4: '家宅', 5: '六畜', 6: '田蠶', 7: '尋人', 8: '行人', 9: '六甲', 10: '山墳', 11: '公訟', 12: '疾病', 13: '失物', 14: '移徙'}
test text[0]: 跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。
test labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [ ]:
v1_use_tokenizer = BertTokenizer.from_pretrained("./fine-tuned-bert")
v1_use_model = BertForSequenceClassification.from_pretrained(
    "./fine-tuned-bert")

In [ ]:
for text in TEST_SENTENCES:
    print(f"{text}:{'='*10}")
    top_labels = predict_top_k(
        tokenizer=v1_use_tokenizer,
        model=v1_use_model,
        text=text,
        top_k=3
    )
    for id, score in top_labels:
        label = test_label_to_id[id]
        print(f"Label: {label}, Score: {score:.4f}")

這是一段與感情相關的提問:==========
Label: 婚姻, Score: 0.1919
Label: 家宅, Score: 0.1358
Label: 行人, Score: 0.0847
求財運:==========
Label: 自身, Score: 0.3014
Label: 交易, Score: 0.0787
Label: 疾病, Score: 0.0613
家人生病:==========
Label: 家宅, Score: 0.1779
Label: 移徙, Score: 0.1030
Label: 六畜, Score: 0.0741
工作、升遷:==========
Label: 家宅, Score: 0.1311
Label: 交易, Score: 0.1201
Label: 移徙, Score: 0.1021


## v2 ckiplab/bert-base-chinese

Provide more training data

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments

labels = DIVINE_LABELS.keys()

v2_trainer = BERTTrainer(
    model_name="ckiplab/bert-base-chinese",
    labels=list(labels),
    divine_data=聖意Doc,
    stopwords=stopwords
)

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded 2298 stopwords from stopwords.txt


In [ ]:
# OOM in CPU mode
# v2_trainer.train()

v2_training_args = TrainingArguments(
    run_name="v2",
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=1,
)

v2_trainer.train(training_args=v2_training_args)
# cat ./logs/trainer_state.json

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training BERT model...


model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,2.653104
2,No log,2.562119
3,No log,2.478492
4,No log,2.396440
5,No log,2.280721
6,No log,2.180317
7,No log,2.051016
8,No log,1.895834
9,No log,1.766090
10,2.339600,1.652857


Training completed.


In [ ]:
# v2_train_model.save_pretrained("./fine-tuned-bert")
# v2_train_tokenizer.save_pretrained("./fine-tuned-bert")
v2_trainer.save_model()

Saving fine-tuned BERT model to ./fine-tuned-bert...
Fine-tuned BERT model saved to ./fine-tuned-bert
Tokenizer saved to ./fine-tuned-bert


In [ ]:
# Double check expected label and id mapping

test_label_to_id = {label: idx for idx,
                    label in enumerate(DIVINE_LABELS.keys())}
print('test label_to_id:', test_label_to_id)
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}
print('test id_to_label:', test_id_to_label)
print('test text[0]:', ['\n'.join(docs) for docs in DIVINE_Q.values()][0])
print('test labels:', [test_label_to_id[label]
      for label in DIVINE_LABELS.keys()])

test label_to_id: {'交易': 0, '婚姻': 1, '求財': 2, '自身': 3, '家宅': 4, '六畜': 5, '田蠶': 6, '尋人': 7, '行人': 8, '六甲': 9, '山墳': 10, '公訟': 11, '疾病': 12, '失物': 13, '移徙': 14}
test id_to_label: {0: '交易', 1: '婚姻', 2: '求財', 3: '自身', 4: '家宅', 5: '六畜', 6: '田蠶', 7: '尋人', 8: '行人', 9: '六甲', 10: '山墳', 11: '公訟', 12: '疾病', 13: '失物', 14: '移徙'}
test text[0]: 跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。
test labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


### Predict - BertClassification

In [ ]:
v2_use_tokenizer = BertTokenizer.from_pretrained("./fine-tuned-bert")
v2_use_model = BertForSequenceClassification.from_pretrained(
    "./fine-tuned-bert")

In [ ]:
# Predicting
for text in TEST_SENTENCES:
    print(f"{text}:{'='*10}")
    top_labels = predict_top_k(
        tokenizer=v2_use_tokenizer,
        model=v2_use_model,
        text=text,
        top_k=3
    )
    for id, score in top_labels:
        label = v2_trainer.id_to_label[id]
        print(f"Label: {label}, Score: {score:.4f}")

這是一段與感情相關的提問:==========
Label: 婚姻, Score: 0.3369
Label: 田蠶, Score: 0.0737
Label: 公訟, Score: 0.0722
求財運:==========
Label: 求財, Score: 0.4355
Label: 山墳, Score: 0.1026
Label: 家宅, Score: 0.0856
家人生病:==========
Label: 疾病, Score: 0.2084
Label: 家宅, Score: 0.1029
Label: 移徙, Score: 0.0779
工作、升遷:==========
Label: 移徙, Score: 0.1432
Label: 田蠶, Score: 0.1084
Label: 疾病, Score: 0.0819


### Predict - SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer

v2_sequence_model = SentenceTransformer('./fine-tuned-bert')

In [ ]:
test_model_similarity_on_divine(v2_sequence_model)

這是一段與感情相關的提問:==========
Label: 疾病, Score: 0.4218
Label: 婚姻, Score: 0.4105
Label: 求財, Score: 0.3415
求財運:==========
Label: 求財, Score: 0.8188
Label: 失物, Score: 0.3252
Label: 家宅, Score: 0.2961
家人生病:==========
Label: 疾病, Score: 0.6642
Label: 婚姻, Score: 0.4274
Label: 家宅, Score: 0.4214
工作、升遷:==========
Label: 六甲, Score: 0.4494
Label: 田蠶, Score: 0.4494
Label: 行人, Score: 0.4493


## v3 ckiplab/bert-base-chinese

Split train/val sets

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments

labels = DIVINE_LABELS.keys()

v3_trainer = BERTTrainer(
    model_name="ckiplab/bert-base-chinese",
    labels=list(labels),
    divine_data=DIVINE_TRAIN_DATA,
    stopwords=stopwords
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded 2298 stopwords from stopwords.txt


In [ ]:
# OOM in CPU mode
# v3_trainer.train()

v3_training_args = TrainingArguments(
    run_name="v3",
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,  # 2e-5
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,  # 200
    warmup_ratio=0.1,  # Learning rate warmup
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=2,  # 1
    save_strategy="epoch",
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,  # for early stop
)

v3_trainer.train(training_args=v3_training_args)
# cat ./logs/trainer_state.json

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training BERT model...


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,2.837506
2,No log,2.807978
3,No log,2.850493
4,No log,2.921801
5,No log,3.009323
6,No log,3.151917
7,No log,3.277641
8,No log,3.342663
9,No log,3.443507
10,1.296900,3.522393


Training completed.


In [ ]:
v3_trainer.save_model()

Saving fine-tuned BERT model to ./fine-tuned-bert...
Fine-tuned BERT model saved to ./fine-tuned-bert
Tokenizer saved to ./fine-tuned-bert


In [ ]:
# Double check expected label and id mapping

test_label_to_id = {label: idx for idx,
                    label in enumerate(DIVINE_LABELS.keys())}
print('test label_to_id:', test_label_to_id)
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}
print('test id_to_label:', test_id_to_label)
print('test text[0]:', ['\n'.join(docs) for docs in DIVINE_Q.values()][0])
print('test labels:', [test_label_to_id[label]
      for label in DIVINE_LABELS.keys()])

test label_to_id: {'交易': 0, '婚姻': 1, '求財': 2, '自身': 3, '家宅': 4, '六畜': 5, '田蠶': 6, '尋人': 7, '行人': 8, '六甲': 9, '山墳': 10, '公訟': 11, '疾病': 12, '失物': 13, '移徙': 14}
test id_to_label: {0: '交易', 1: '婚姻', 2: '求財', 3: '自身', 4: '家宅', 5: '六畜', 6: '田蠶', 7: '尋人', 8: '行人', 9: '六甲', 10: '山墳', 11: '公訟', 12: '疾病', 13: '失物', 14: '移徙'}
test text[0]: 跟地緣有關，艋舺龍山寺過去是商業重地，廟方的運營管理也常是商界人士，來廟裡卜問交易的事情，應該對於這個區域生活的人們，是在切身不過的重要項目了，排在第一個也可見當地信眾的重視。
test labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


### Predict - BertClassification

In [ ]:
v3_use_tokenizer = BertTokenizer.from_pretrained("./fine-tuned-bert")
v3_use_model = BertForSequenceClassification.from_pretrained(
    "./fine-tuned-bert")

In [ ]:
for text in TEST_SENTENCES:
    print(f"{text}:{'='*10}")
    top_labels = predict_top_k(
        tokenizer=v3_use_tokenizer,
        model=v3_use_model,
        text=text,
        top_k=3
    )
    for id, score in top_labels:
        label = v3_trainer.id_to_label[id]
        print(f"Label: {label}, Score: {score:.4f}")

這是一段與感情相關的提問:==========
Label: 婚姻, Score: 0.3369
Label: 田蠶, Score: 0.0737
Label: 公訟, Score: 0.0722
求財運:==========
Label: 求財, Score: 0.4355
Label: 山墳, Score: 0.1026
Label: 家宅, Score: 0.0856
家人生病:==========
Label: 疾病, Score: 0.2084
Label: 家宅, Score: 0.1029
Label: 移徙, Score: 0.0779
工作、升遷:==========
Label: 移徙, Score: 0.1432
Label: 田蠶, Score: 0.1084
Label: 疾病, Score: 0.0819


### Predict - SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer

v3_sequence_model = SentenceTransformer('./fine-tuned-bert')

In [ ]:
test_model_similarity_on_divine(v3_sequence_model)

這是一段與感情相關的提問:==========
Label: 疾病, Score: 0.4218
Label: 婚姻, Score: 0.4105
Label: 求財, Score: 0.3415
求財運:==========
Label: 求財, Score: 0.8188
Label: 失物, Score: 0.3252
Label: 家宅, Score: 0.2961
家人生病:==========
Label: 疾病, Score: 0.6642
Label: 婚姻, Score: 0.4274
Label: 家宅, Score: 0.4214
工作、升遷:==========
Label: 六甲, Score: 0.4494
Label: 田蠶, Score: 0.4494
Label: 行人, Score: 0.4493


## v4 ckiplab/bert-base-chinese

Introduce data augmentation

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments

labels = DIVINE_LABELS.keys()

v4_trainer = BERTTrainer(
    model_name="ckiplab/bert-base-chinese",
    labels=list(labels),
    divine_data=all_divine,
    stopwords=stopwords
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded 2298 stopwords from stopwords.txt


In [ ]:
v4_trainer.run_augmentation()

Augmented texts: ['是否會因為拖延而失去交易機會?', '是否會因為拖延而失去交易機會?關於商業的困擾']
Augmented texts: ['是否能在期限內完成交易?', '是否能在期限內完成交易?關於買賣的困擾', '是否能在期限內完成交易?希望解決成交的問題']
Augmented texts: ['此次買賣是否能帶來高額利潤?', '此次買賣是否能帶來高額利潤?']
Augmented texts: ['是否需要擴大合作範圍來提高效益?', '是否需要擴大合作範圍來提高效益?']
Augmented texts: ['如何確保在交易中達到雙贏?希望解決交易的問題', '如何確保在交易中達到雙贏?']
Augmented texts: ['是否應該為這次交易組建專門的團隊?']
Augmented texts: ['合作夥伴是否會影響交易結果?']
Augmented texts: ['是否需要與政府部門合作來促成交易?']
Augmented texts: ['交易進程是否會因人為因素而延遲?關於交易的困擾', '交易進程是否會因人為因素而延遲?']
Augmented texts: ['是否應該探索更有創意的合作方式?']
Augmented texts: ['是否會因交易成本過高而導致虧損?詢問成交的未來', '是否會因交易成本過高而導致虧損?']
Augmented texts: ['此交易是否會因突發事件而受挫?關於成交的困擾', '此交易是否會因突發事件而受挫?', '此交易是否會因突發事件而受挫?']
Augmented texts: ['是否需要修改交易計劃以適應市場變化?關於交易的困擾', '是否需要修改交易計劃以適應市場變化?']
Augmented texts: ['交易過程中的阻礙是否能克服?', '交易過程中的阻礙是否能克服?']
Augmented texts: ['是否需要更多資金來完成交易?']
Augmented texts: ['是否需要請專業人士參與談判?', '是否需要請專業人士參與談判?詢問交易的未來']
Augmented texts: ['是否需要額外的融資支持以完成交易?', '是否需要額外的融資支持以完成交易?']
Augmented texts: ['是否應該在交易前進行更多談判?關於買賣

In [ ]:
# OOM in CPU mode
# v4_trainer.train()

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

v4_training_args = TrainingArguments(
    run_name="v4",
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,  # Limit to keep only 2 most recent checkpoints
    # eval_strategy="steps", # epoch
    # eval_steps=100,
    learning_rate=1e-5,  # 2e-5
    per_device_train_batch_size=8,  # 16
    per_device_eval_batch_size=8,  # 16
    num_train_epochs=100,  # 200
    warmup_ratio=0.2,  # Learning rate warmup
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=1,  # 1
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,  # for early stop
    # callbacks=[early_stopping_callback]
)

v4_trainer.train(training_args=v4_training_args)
# cat ./logs/trainer_state.json

Training BERT model...


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Splitting dataset...


Epoch,Training Loss,Validation Loss
1,No log,2.779303
2,No log,2.791217
3,No log,2.811333
4,No log,2.838557
5,2.676900,2.871284
6,2.676900,2.913309
7,2.676900,2.964036
8,2.676900,3.022477
9,2.676900,3.088237
10,2.596800,3.158079


Training completed.


In [ ]:
v4_trainer.save_model()

Saving fine-tuned BERT model to ./fine-tuned-bert...
Fine-tuned BERT model saved to ./fine-tuned-bert
Tokenizer saved to ./fine-tuned-bert


In [ ]:
# Double check expected label and id mapping

test_label_to_id = {label: idx for idx, label in enumerate(all_divine.keys())}
print('test label_to_id:', test_label_to_id)
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}
print('test id_to_label:', test_id_to_label)
print('test text[0]:', ['\n'.join(docs) for docs in all_divine.values()][0])
print('test labels:', [test_label_to_id[label]
      for label in DIVINE_LABELS.keys()])

test label_to_id: {'交易': 0, '婚姻': 1, '求財': 2, '自身': 3, '家宅': 4, '六畜': 5, '田蠶': 6, '尋人': 7, '行人': 8, '六甲': 9, '山墳': 10, '公訟': 11, '疾病': 12, '失物': 13, '移徙': 14}
test id_to_label: {0: '交易', 1: '婚姻', 2: '求財', 3: '自身', 4: '家宅', 5: '六畜', 6: '田蠶', 7: '尋人', 8: '行人', 9: '六甲', 10: '山墳', 11: '公訟', 12: '疾病', 13: '失物', 14: '移徙'}
test text[0]: 是否會因為拖延而失去交易機會?
是否能在期限內完成交易?
此次買賣是否能帶來高額利潤?
是否需要擴大合作範圍來提高效益?
如何確保在交易中達到雙贏?
是否應該為這次交易組建專門的團隊?
合作夥伴是否會影響交易結果?
是否需要與政府部門合作來促成交易?
交易進程是否會因人為因素而延遲?
是否應該探索更有創意的合作方式?
是否會因交易成本過高而導致虧損?
此交易是否會因突發事件而受挫?
是否需要修改交易計劃以適應市場變化?
交易過程中的阻礙是否能克服?
是否需要更多資金來完成交易?
是否需要請專業人士參與談判?
是否需要額外的融資支持以完成交易?
是否應該在交易前進行更多談判?
這次合作是否能提升市場份額?
此次交易是否會增強市場地位?
是否需要提前準備交易的應急方案?
此次交易是否會對現有資產造成壓力?
是否需要調整產品價格以吸引客戶?
是否應該更快行動以擊敗競爭對手?
合同的細節是否需要進一步確認?
此次交易的成功率是否足夠高?
如何應對交易中的突發情況?
合作過程中是否需要額外的技術支持?
是否需要更多準備以降低風險?
交易是否會受到政策變化的影響?
是否有其他替代供應商可以選擇?
市場環境是否支持這次交易?
是否會因市場需求變化而改變交易內容?
是否需要額外的保證措施?
此次交易是否符合行業規範和標準?
此交易是否會因時機不當而失敗?
是否需要更多時間來準備交易細節?
我們的產品是否能滿足對方的需求?
是否應該暫緩交易計劃?
交易的預期利潤是否合理?
是否應該在競爭中更積極推進交易?
如何提升雙方在合作中的滿意

### Predict - BertClassification

In [ ]:
v4_use_tokenizer = BertTokenizer.from_pretrained("./fine-tuned-bert")
v4_use_model = BertForSequenceClassification.from_pretrained(
    "./fine-tuned-bert")

In [ ]:
for text in TEST_SENTENCES:
    print(f"{text}:{'='*10}")
    top_labels = predict_top_k(
        tokenizer=v4_use_tokenizer,
        model=v4_use_model,
        text=text,
        top_k=3
    )
    for id, score in top_labels:
        label = v4_trainer.id_to_label[id]
        print(f"Label: {label}, Score: {score:.4f}")

這是一段與感情相關的提問:==========
Label: 求財, Score: 0.0964
Label: 家宅, Score: 0.0859
Label: 婚姻, Score: 0.0833
求財運:==========
Label: 疾病, Score: 0.0979
Label: 求財, Score: 0.0799
Label: 六畜, Score: 0.0780
家人生病:==========
Label: 家宅, Score: 0.0987
Label: 疾病, Score: 0.0892
Label: 移徙, Score: 0.0778
工作、升遷:==========
Label: 疾病, Score: 0.0951
Label: 尋人, Score: 0.0776
Label: 求財, Score: 0.0756


### Predict - SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer

v4_sequence_model = SentenceTransformer('./fine-tuned-bert')

In [ ]:
test_model_similarity_on_divine(v4_sequence_model)

這是一段與感情相關的提問:==========
Label: 婚姻, Score: 0.5320
Label: 疾病, Score: 0.5233
Label: 尋人, Score: 0.5064
求財運:==========
Label: 求財, Score: 0.8566
Label: 家宅, Score: 0.5942
Label: 失物, Score: 0.5853
家人生病:==========
Label: 疾病, Score: 0.7393
Label: 尋人, Score: 0.5723
Label: 婚姻, Score: 0.5671
工作、升遷:==========
Label: 自身, Score: 0.6674
Label: 家宅, Score: 0.6558
Label: 疾病, Score: 0.6511


## v5 ckiplab/bert-base-chinese

Improve data quality by adding LLM Q&A texts

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments

labels = DIVINE_LABELS.keys()

v5_trainer = BERTTrainer(
    model_name="ckiplab/bert-base-chinese",
    labels=list(labels),
    divine_data=all_divine,
    stopwords=stopwords
)

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded 2298 stopwords from stopwords.txt


In [ ]:
v5_trainer.run_augmentation()

Augmented texts: ['是否會因為拖延而失去交易機會?']
Augmented texts: ['是否能在期限內完成交易?', '是否能在期限內完成交易?']
Augmented texts: ['此次買賣是否能帶來高額利潤?']
Augmented texts: ['是否需要擴大合作範圍來提高效益?希望解決交易的問題', '是否需要擴大合作範圍來提高效益?', '是否需要擴大合作範圍來提高效益?']
Augmented texts: ['如何確保在交易中達到雙贏?']
Augmented texts: ['是否應該為這次交易組建專門的團隊?']
Augmented texts: ['合作夥伴是否會影響交易結果?', '合作夥伴是否會影響交易結果?關於協議的困擾']
Augmented texts: ['是否需要與政府部門合作來促成交易?關於商業的困擾', '是否需要與政府部門合作來促成交易?', '是否需要與政府部門合作來促成交易?希望解決商業的問題']
Augmented texts: ['交易進程是否會因人為因素而延遲?', '交易進程是否會因人為因素而延遲?']
Augmented texts: ['是否應該探索更有創意的合作方式?', '是否應該探索更有創意的合作方式?希望解決談判的問題']
Augmented texts: ['是否會因交易成本過高而導致虧損?']
Augmented texts: ['此交易是否會因突發事件而受挫?關於協議的困擾', '此交易是否會因突發事件而受挫?']
Augmented texts: ['是否需要修改交易計劃以適應市場變化?', '是否需要修改交易計劃以適應市場變化?']
Augmented texts: ['交易過程中的阻礙是否能克服?']
Augmented texts: ['是否需要更多資金來完成交易?']
Augmented texts: ['是否需要請專業人士參與談判?', '是否需要請專業人士參與談判?']
Augmented texts: ['是否需要額外的融資支持以完成交易?', '是否需要額外的融資支持以完成交易?']
Augmented texts: ['是否應該在交易前進行更多談判?希望解決合作的問題', '是否應該在交易前進行更多談判?']
Augmented texts: ['

In [ ]:
# OOM in CPU mode
# v4_trainer.train()

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

v5_training_args = TrainingArguments(
    run_name="v5-single-label",
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,  # Limit to keep only 2 most recent checkpoints
    # eval_strategy="steps", # epoch
    # eval_steps=100,
    learning_rate=1e-5,  # 2e-5
    per_device_train_batch_size=8,  # 16
    per_device_eval_batch_size=8,  # 16
    num_train_epochs=100,  # 100
    warmup_ratio=0.1,  # Learning rate warmup
    weight_decay=0.05,  # 0.01
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=4,  # 1
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,  # for early stop
    # callbacks=[early_stopping_callback]
)

v5_trainer.train(training_args=v5_training_args)
# cat ./logs/trainer_state.json

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training BERT model...


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Splitting dataset...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: osisdie (osisdie-kevinw). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,2.811706
2,No log,2.838004
3,No log,2.890278
4,No log,2.960756
5,No log,3.045115
6,No log,3.131421
7,No log,3.214489
8,No log,3.293887
9,No log,3.360221
10,1.290800,3.414594


Training completed.


In [ ]:
v5_trainer.save_model()

Saving fine-tuned BERT model to ./fine-tuned-bert...
Fine-tuned BERT model saved to ./fine-tuned-bert
Tokenizer saved to ./fine-tuned-bert


In [ ]:
# Double check expected label and id mapping

from itertools import chain

test_label_to_id = {label: idx for idx,
                    label in enumerate(DIVINE_LABELS.keys())}
print('test label_to_id:', test_label_to_id)
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}
print('test id_to_label:', test_id_to_label)
print('test text[0]:', ['\n'.join(docs) for docs in all_divine.values()][0])
print('test labels:', [test_label_to_id[label]
      for label in DIVINE_LABELS.keys()])

test label_to_id: {'交易': 0, '婚姻': 1, '求財': 2, '自身': 3, '家宅': 4, '六畜': 5, '田蠶': 6, '尋人': 7, '行人': 8, '六甲': 9, '山墳': 10, '公訟': 11, '疾病': 12, '失物': 13, '移徙': 14}
test id_to_label: {0: '交易', 1: '婚姻', 2: '求財', 3: '自身', 4: '家宅', 5: '六畜', 6: '田蠶', 7: '尋人', 8: '行人', 9: '六甲', 10: '山墳', 11: '公訟', 12: '疾病', 13: '失物', 14: '移徙'}
test text[0]: 是否應該尋求更有利的合作夥伴?
是否需要等待更好的時機進行交易?
這次合作是否會對未來的策略有影響?
這次合作的成本是否能控制在預算範圍內?
這筆交易是否會成為公司的轉折點?
與對方合作是否會帶來長期利益?
是否會有外部競爭者插手交易?
是否需要調整策略以適應交易需求?
是否需要提前安排好貨物運輸計劃?
是否應該提前結束談判以快速成交?
如何確保交易過程中的數據安全?
如何確保在交易中達到雙贏?
這次合同是否會順利簽訂?
是否需要根據對方需求調整供應計劃?
這次合作是否有助於進軍新市場?
是否需要為這次交易制定專門的培訓計劃?
此次交易是否會增強市場地位?
市場波動是否會影響交易結果?
是否需要調整交易條件以達成協議?
是否需要重新評估對方的市場地位?
是否應該邀請法律顧問參與談判?
是否需要請專業人士參與談判?
是否需要額外的融資支持以完成交易?
此交易是否會導致資源過度分散?
此交易是否會影響公司的核心業務?
如果交易失敗，是否有替代方案?
交易的規模是否需要縮小以降低風險?
是否需要加強交易過程中的監管措施?
是否需要引入第三方監督交易?
是否應該提前制定風險應對計劃?
是否應該選擇更穩妥的合作方案?
是否需要與對方分享更多數據和資源?
是否有需要提前解決的物流問題?
是否應該尋求專家的建議來完成交易?
如何解決交易中的分歧?
交易中的潛在問題是否可以提前發現?
是否會因市場需求變化而改變交易內容?
是否能在期限內完成交易?
如何設計更具吸引力的交易條件?
此交易是否能促進雙方的互利共贏?
我們的報價是否能吸

### Predict - BertClassification

In [ ]:
v5_use_tokenizer = BertTokenizer.from_pretrained("./fine-tuned-bert")
v5_use_model = BertForSequenceClassification.from_pretrained(
    "./fine-tuned-bert")

In [ ]:
for text in TEST_SENTENCES:
    print(f"{text}:{'='*10}")
    top_labels = predict_top_k(
        tokenizer=v5_use_tokenizer,
        model=v5_use_model,
        text=text,
        top_k=3
    )
    for id, score in top_labels:
        label = v5_trainer.id_to_label[id]
        print(f"Label: {label}, Score: {score:.4f}")

這是一段與感情相關的提問:==========
Label: 家宅, Score: 0.0852
Label: 行人, Score: 0.0849
Label: 公訟, Score: 0.0793
求財運:==========
Label: 行人, Score: 0.1029
Label: 公訟, Score: 0.0921
Label: 失物, Score: 0.0804
家人生病:==========
Label: 公訟, Score: 0.0833
Label: 交易, Score: 0.0829
Label: 行人, Score: 0.0761
工作、升遷:==========
Label: 失物, Score: 0.0864
Label: 六甲, Score: 0.0825
Label: 行人, Score: 0.0801


### Predict - SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer

v5_sequence_model = SentenceTransformer('./fine-tuned-bert')

In [ ]:
test_model_similarity_on_divine(v5_sequence_model)

這是一段與感情相關的提問:==========
Label: 婚姻, Score: 0.5320
Label: 疾病, Score: 0.5233
Label: 尋人, Score: 0.5064
求財運:==========
Label: 求財, Score: 0.8566
Label: 家宅, Score: 0.5942
Label: 失物, Score: 0.5853
家人生病:==========
Label: 疾病, Score: 0.7393
Label: 尋人, Score: 0.5723
Label: 婚姻, Score: 0.5671
工作、升遷:==========
Label: 自身, Score: 0.6704
Label: 家宅, Score: 0.6558
Label: 疾病, Score: 0.6511


## v6 ckiplab/bert-base-chinese

Change to multiple label, 50 epochs

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments

labels = DIVINE_LABELS.keys()

v6_trainer = BERTMultiLabelTrainer(
    model_name="ckiplab/bert-base-chinese",
    labels=list(labels),
    divine_data=labeled_data,
    stopwords=stopwords
)

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of BertForMultiLabelClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# OOM in CPU mode
# v4_trainer.train()

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

v6_training_args = TrainingArguments(
    run_name="v6-multi-label",
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,  # Limit to keep only 2 most recent checkpoints
    # eval_strategy="steps", # epoch
    # eval_steps=100,
    learning_rate=2e-5,  # 2e-5
    per_device_train_batch_size=8,  # 16
    per_device_eval_batch_size=8,  # 16
    num_train_epochs=50,  # 100
    warmup_ratio=0.1,  # Learning rate warmup
    weight_decay=0.05,  # 0.01
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=4,  # 1
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,  # for early stop
    # callbacks=[early_stopping_callback]
)

v6_trainer.train(training_args=v6_training_args)
# cat ./logs/trainer_state.json

Training BERT model...


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Splitting dataset...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: osisdie (osisdie-kevinw). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.705107
2,No log,0.699106
3,No log,0.686923
4,No log,0.683993
5,No log,0.679769
6,No log,0.675281
7,No log,0.685944
8,No log,0.688845
9,No log,0.684507
10,0.318400,0.683768


Training completed.


In [ ]:
v6_trainer.save_model()

Saving fine-tuned BERT model to ./fine-tuned-bert...
Fine-tuned BERT model saved to ./fine-tuned-bert
Tokenizer saved to ./fine-tuned-bert


In [ ]:
# Double check expected label and id mapping

test_label_to_id = {label: idx for idx,
                    label in enumerate(DIVINE_LABELS.keys())}
print('test label_to_id:', test_label_to_id)
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}
print('test id_to_label:', test_id_to_label)
print('test text[0]:', ['\n'.join(docs) for docs in labeled_data.values()][0])
print('test labels:', [test_label_to_id[label]
      for label in DIVINE_LABELS.keys()])

test label_to_id: {'交易': 0, '婚姻': 1, '求財': 2, '自身': 3, '家宅': 4, '六畜': 5, '田蠶': 6, '尋人': 7, '行人': 8, '六甲': 9, '山墳': 10, '公訟': 11, '疾病': 12, '失物': 13, '移徙': 14}
test id_to_label: {0: '交易', 1: '婚姻', 2: '求財', 3: '自身', 4: '家宅', 5: '六畜', 6: '田蠶', 7: '尋人', 8: '行人', 9: '六甲', 10: '山墳', 11: '公訟', 12: '疾病', 13: '失物', 14: '移徙'}
test text[0]: 是否應該尋求更有利的合作夥伴?
是否需要等待更好的時機進行交易?
這次合作是否會對未來的策略有影響?
這次合作的成本是否能控制在預算範圍內?
這筆交易是否會成為公司的轉折點?
與對方合作是否會帶來長期利益?
是否會有外部競爭者插手交易?
是否需要調整策略以適應交易需求?
是否需要提前安排好貨物運輸計劃?
是否應該提前結束談判以快速成交?
如何確保交易過程中的數據安全?
如何確保在交易中達到雙贏?
這次合同是否會順利簽訂?
是否需要根據對方需求調整供應計劃?
這次合作是否有助於進軍新市場?
是否需要為這次交易制定專門的培訓計劃?
此次交易是否會增強市場地位?
市場波動是否會影響交易結果?
是否需要調整交易條件以達成協議?
是否需要重新評估對方的市場地位?
是否應該邀請法律顧問參與談判?
是否需要請專業人士參與談判?
是否需要額外的融資支持以完成交易?
此交易是否會導致資源過度分散?
此交易是否會影響公司的核心業務?
如果交易失敗，是否有替代方案?
交易的規模是否需要縮小以降低風險?
是否需要加強交易過程中的監管措施?
是否需要引入第三方監督交易?
是否應該提前制定風險應對計劃?
是否應該選擇更穩妥的合作方案?
是否需要與對方分享更多數據和資源?
是否有需要提前解決的物流問題?
是否應該尋求專家的建議來完成交易?
如何解決交易中的分歧?
交易中的潛在問題是否可以提前發現?
是否會因市場需求變化而改變交易內容?
是否能在期限內完成交易?
如何設計更具吸引力的交易條件?
此交易是否能促進雙方的互利共贏?
我們的報價是否能吸

### Predict - BertClassification

In [ ]:
v6_use_tokenizer = BertTokenizer.from_pretrained("./fine-tuned-bert")
v6_use_model = BertForSequenceClassification.from_pretrained(
    "./fine-tuned-bert")

In [ ]:
# Testing the Model
# inputs = tokenizer("Sample input text", return_tensors="pt")
# outputs = model(**inputs)
# predictions = outputs["logits"] > 0.5  # Threshold at 0.5
# print(predictions)

predicted_class_ids = predict_threshold(
    v6_use_tokenizer, v6_use_model, TEST_SENTENCES[0], threshold=0.1)
print(predicted_class_ids)

for text in TEST_SENTENCES:
    print(f"{text}:{'='*10}")
    top_labels = predict_top_k_threshold(
        tokenizer=v6_use_tokenizer,
        model=v6_use_model,
        text=text,
        top_k=3,
        threshold=0.1
    )
    for id, score in top_labels:
        label = v6_trainer.id_to_label[id]
        print(f"Label: {label}, Score: {score:.4f}")

[(4, 0.5446382164955139)]
這是一段與感情相關的提問:==========
Label: 家宅, Score: 0.5446
Label: 移徙, Score: 0.3196
Label: 求財, Score: 0.2391
求財運:==========
Label: 移徙, Score: 0.5232
Label: 自身, Score: 0.1282
Label: 山墳, Score: 0.1056
家人生病:==========
Label: 移徙, Score: 0.4744
Label: 自身, Score: 0.3964
Label: 疾病, Score: 0.2497
工作、升遷:==========
Label: 移徙, Score: 0.3896
Label: 山墳, Score: 0.1899
Label: 家宅, Score: 0.1760


### Predict - SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer

v6_sequence_model = SentenceTransformer('./fine-tuned-bert')

In [ ]:
test_model_similarity_on_divine(v6_sequence_model)

這是一段與感情相關的提問:==========
Label: 疾病, Score: 0.5254
Label: 婚姻, Score: 0.5166
Label: 尋人, Score: 0.5075
求財運:==========
Label: 求財, Score: 0.8464
Label: 家宅, Score: 0.5849
Label: 失物, Score: 0.5737
家人生病:==========
Label: 疾病, Score: 0.7418
Label: 婚姻, Score: 0.5514
Label: 家宅, Score: 0.5403
工作、升遷:==========
Label: 家宅, Score: 0.6318
Label: 田蠶, Score: 0.6259
Label: 尋人, Score: 0.6226


## v6-2 ckiplab/bert-base-chinese

Change to multiple label, enable data augmentation, 100 epochs

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments

labels = DIVINE_LABELS.keys()

v6_2_trainer = BERTMultiLabelTrainer(
    model_name="ckiplab/bert-base-chinese",
    labels=list(labels),
    divine_data=labeled_data,
    stopwords=stopwords
)

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of BertForMultiLabelClassification were not initialized from the model checkpoint at ckiplab/bert-base-chinese and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
v6_2_trainer.run_augmentation()

Augmented texts: ['這筆交易是否會成為公司的轉折點?']
Augmented texts: ['如何設計更具吸引力的交易條件?']
Augmented texts: ['合作協議的條款是否清晰明確?']
Augmented texts: ['是否需要與法律顧問再次核實合同?']
Augmented texts: ['是否應該制定詳細的交易後續計劃?']
Augmented texts: ['是否應該提前準備替代方案?']
Augmented texts: ['這筆交易是否符合公司的長期發展目標?']
Augmented texts: ['是否需要更多資金來支持這筆交易?']
Augmented texts: ['是否需要進行額外的背景調查?']
Augmented texts: ['是否會因交易而失去其他發展機會?']
Augmented texts: ['是否會因匯率波動而影響交易結果?']
Augmented texts: ['是否需要設立明確的交易指標以衡量成功?']
Augmented texts: ['如何應對交易中的突發情況?']
Augmented texts: ['交易的目標是否與公司的核心價值一致?']
Augmented texts: ['對方的信用是否值得信任?']
Augmented texts: ['如何確保在交易中達到雙贏?']
Augmented texts: ['是否有其他競爭者在爭奪這筆交易?']
Augmented texts: ['是否能在交易中提升品牌影響力?']
Augmented texts: ['是否會因競爭對手而失去交易機會?']
Augmented texts: ['如何說服對方接受我們的條件?']
Augmented texts: ['是否應該尋求銀行或金融機構的支持?']
Augmented texts: ['與這位交易對象的合作歷史是否良好?']
Augmented texts: ['這次合作是否需要額外的資金支持?']
Augmented texts: ['這筆交易的最大風險在哪裡?']
Augmented texts: ['是否應該尋求專家的建議來完成交易?']
Augmented texts: ['是否需要在交易協議中加入退出條款?']
Augmented texts: ['是否需要為這

{'交易:1.0,求財:0.8,田蠶:0.5,六畜:0.4,行人:0.3': ['這筆交易是否會成為公司的轉折點?',
  '如何設計更具吸引力的交易條件?',
  '合作協議的條款是否清晰明確?',
  '是否需要與法律顧問再次核實合同?',
  '是否應該制定詳細的交易後續計劃?',
  '是否應該提前準備替代方案?',
  '這筆交易是否符合公司的長期發展目標?',
  '是否需要更多資金來支持這筆交易?',
  '是否需要進行額外的背景調查?',
  '是否會因交易而失去其他發展機會?',
  '是否會因匯率波動而影響交易結果?',
  '是否需要設立明確的交易指標以衡量成功?',
  '如何應對交易中的突發情況?',
  '交易的目標是否與公司的核心價值一致?',
  '對方的信用是否值得信任?',
  '如何確保在交易中達到雙贏?',
  '是否有其他競爭者在爭奪這筆交易?',
  '是否能在交易中提升品牌影響力?',
  '是否會因競爭對手而失去交易機會?',
  '如何說服對方接受我們的條件?',
  '是否應該尋求銀行或金融機構的支持?',
  '與這位交易對象的合作歷史是否良好?',
  '這次合作是否需要額外的資金支持?',
  '這筆交易的最大風險在哪裡?',
  '是否應該尋求專家的建議來完成交易?',
  '是否需要在交易協議中加入退出條款?',
  '是否需要為這次交易制定專門的培訓計劃?',
  '是否適合現在就啟動交易流程?',
  '是否需要與政府部門合作來促成交易?',
  '這次交易的關鍵點在哪裡?',
  '如何提升雙方在合作中的滿意度?',
  '是否需要尋求更多的談判籌碼?',
  '是否需要額外的保證措施?',
  '交易對象的需求是否完全滿足?',
  '是否有足夠的資源來支持此次交易?',
  '是否應該與對方建立更長期的合作計劃?',
  '如何減少交易過程中的成本支出?',
  '交易的風險是否值得承擔?',
  '是否需要與對方重新談判合作條件?',
  '是否應該在競爭中更積極推進交易?',
  '是否需要更多內部支持以完成交易?',
  '交易條件是否需要重新評估?',
  '是否有潛在的法律問題需要關注?',
  '是否會因市場需求變化而改變交易內容?',
  '是否應該尋求更有利的合作夥伴?',
  

In [ ]:
# OOM in CPU mode
# v4_trainer.train()

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

v6_2_training_args = TrainingArguments(
    run_name="v6-multi-label",
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,  # Limit to keep only 2 most recent checkpoints
    # eval_strategy="steps", # epoch
    # eval_steps=100,
    learning_rate=2e-5,  # 2e-5
    per_device_train_batch_size=8,  # 16
    per_device_eval_batch_size=8,  # 16
    num_train_epochs=100,  # 100
    warmup_ratio=0.1,  # Learning rate warmup
    weight_decay=0.05,  # 0.01
    logging_dir="./logs",
    logging_steps=10,
    gradient_accumulation_steps=4,  # 1
    metric_for_best_model="eval_loss",
    # load_best_model_at_end = True, # for early stop
    # callbacks=[early_stopping_callback]
)

v6_2_trainer.train(training_args=v6_2_training_args)
# cat ./logs/trainer_state.json

Training BERT model...


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Splitting dataset...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: osisdie (osisdie-kevinw). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.694610
2,No log,0.693801
3,No log,0.692715
4,No log,0.691523
5,No log,0.688995
6,No log,0.684723
7,No log,0.676318
8,No log,0.677925
9,No log,0.695524
10,0.324200,0.701224


Training completed.


In [ ]:
v6_2_trainer.save_model()

Saving fine-tuned BERT model to ./fine-tuned-bert...
Fine-tuned BERT model saved to ./fine-tuned-bert
Tokenizer saved to ./fine-tuned-bert


In [ ]:
# Double check expected label and id mapping

test_label_to_id = {label: idx for idx,
                    label in enumerate(DIVINE_LABELS.keys())}
print('test label_to_id:', test_label_to_id)
test_id_to_label = {idx: label for label, idx in test_label_to_id.items()}
print('test id_to_label:', test_id_to_label)
print('test text[0]:', ['\n'.join(docs) for docs in labeled_data.values()][0])
print('test labels:', [test_label_to_id[label]
      for label in DIVINE_LABELS.keys()])

test label_to_id: {'交易': 0, '婚姻': 1, '求財': 2, '自身': 3, '家宅': 4, '六畜': 5, '田蠶': 6, '尋人': 7, '行人': 8, '六甲': 9, '山墳': 10, '公訟': 11, '疾病': 12, '失物': 13, '移徙': 14}
test id_to_label: {0: '交易', 1: '婚姻', 2: '求財', 3: '自身', 4: '家宅', 5: '六畜', 6: '田蠶', 7: '尋人', 8: '行人', 9: '六甲', 10: '山墳', 11: '公訟', 12: '疾病', 13: '失物', 14: '移徙'}
test text[0]: 這筆交易是否會成為公司的轉折點?
如何設計更具吸引力的交易條件?
合作協議的條款是否清晰明確?
是否需要與法律顧問再次核實合同?
是否應該制定詳細的交易後續計劃?
是否應該提前準備替代方案?
這筆交易是否符合公司的長期發展目標?
是否需要更多資金來支持這筆交易?
是否需要進行額外的背景調查?
是否會因交易而失去其他發展機會?
是否會因匯率波動而影響交易結果?
是否需要設立明確的交易指標以衡量成功?
如何應對交易中的突發情況?
交易的目標是否與公司的核心價值一致?
對方的信用是否值得信任?
如何確保在交易中達到雙贏?
是否有其他競爭者在爭奪這筆交易?
是否能在交易中提升品牌影響力?
是否會因競爭對手而失去交易機會?
如何說服對方接受我們的條件?
是否應該尋求銀行或金融機構的支持?
與這位交易對象的合作歷史是否良好?
這次合作是否需要額外的資金支持?
這筆交易的最大風險在哪裡?
是否應該尋求專家的建議來完成交易?
是否需要在交易協議中加入退出條款?
是否需要為這次交易制定專門的培訓計劃?
是否適合現在就啟動交易流程?
是否需要與政府部門合作來促成交易?
這次交易的關鍵點在哪裡?
如何提升雙方在合作中的滿意度?
是否需要尋求更多的談判籌碼?
是否需要額外的保證措施?
交易對象的需求是否完全滿足?
是否有足夠的資源來支持此次交易?
是否應該與對方建立更長期的合作計劃?
如何減少交易過程中的成本支出?
交易的風險是否值得承擔?
是否需要與對方重新談判合作條件?
是否應該在競爭中更積極推進交易?
是否需要更多內部支持以完成交易

### Predict - BertClassification

In [ ]:
v6_2_use_tokenizer = BertTokenizer.from_pretrained("./fine-tuned-bert")
v6_2_use_model = BertForSequenceClassification.from_pretrained(
    "./fine-tuned-bert")

In [ ]:
# Testing the Model
# inputs = tokenizer("Sample input text", return_tensors="pt")
# outputs = model(**inputs)
# predictions = outputs["logits"] > 0.5  # Threshold at 0.5
# print(predictions)

predicted_class_ids = predict_threshold(
    tokenizer=v6_2_use_tokenizer,
    model=v6_2_use_model,
    text=TEST_SENTENCES[0],
    threshold=0.1
)
print(predicted_class_ids)

for text in TEST_SENTENCES:
    print(f"{text}:{'='*10}")
    top_labels = predict_top_k_threshold(
        tokenizer=v6_2_use_tokenizer,
        model=v6_2_use_model,
        text=text,
        top_k=3,
        threshold=0.1
    )
    for id, score in top_labels:
        label = v6_2_trainer.id_to_label[id]
        print(f"Label: {label}, Score: {score:.4f}")

[(4, 0.8541063070297241)]
這是一段與感情相關的提問:==========
Label: 家宅, Score: 0.8541
Label: 自身, Score: 0.3372
Label: 移徙, Score: 0.3102
求財運:==========
Label: 家宅, Score: 0.4154
Label: 移徙, Score: 0.0499
Label: 自身, Score: 0.0308
家人生病:==========
Label: 家宅, Score: 1.1251
Label: 自身, Score: 0.7553
Label: 疾病, Score: 0.0042
工作、升遷:==========
Label: 家宅, Score: 0.6824
Label: 自身, Score: 0.4195
Label: 移徙, Score: 0.3692


### Predict - SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer

v6_2_sequence_model = SentenceTransformer('./fine-tuned-bert')

In [ ]:
test_model_similarity_on_divine(v6_2_sequence_model)

這是一段與感情相關的提問:==========
Label: 婚姻, Score: 0.5089
Label: 疾病, Score: 0.4724
Label: 尋人, Score: 0.4575
求財運:==========
Label: 求財, Score: 0.8084
Label: 家宅, Score: 0.5507
Label: 失物, Score: 0.5341
家人生病:==========
Label: 疾病, Score: 0.7211
Label: 婚姻, Score: 0.5350
Label: 家宅, Score: 0.4917
工作、升遷:==========
Label: 移徙, Score: 0.6313
Label: 尋人, Score: 0.5586
Label: 疾病, Score: 0.5568


# Upload fine-tuned model

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

!ln -s "/content/drive/MyDrive/Colab Notebooks/" /gdrive
# later on copy fine-tuned model to drive /content/drive/MyDrive/Colab Notebooks/models

Mounted at /content/drive


In [ ]:
# !zip fine-tuned-bert_v6_2.zip -r fine-tuned-bert/

updating: fine-tuned-bert/ (stored 0%)
updating: fine-tuned-bert/vocab.txt (deflated 48%)
updating: fine-tuned-bert/tokenizer_config.json (deflated 75%)
updating: fine-tuned-bert/special_tokens_map.json (deflated 42%)
updating: fine-tuned-bert/model.safetensors (deflated 7%)
updating: fine-tuned-bert/config.json (deflated 61%)


In [ ]:
# !mkdir -p '/content/drive/MyDrive/Colab Notebooks/models'
# !cp '/content/drive/MyDrive/Colab Notebooks/models/fine-tuned-bert_v5.zip' '/content'
# !cp '/content/fine-tuned-bert_v6_2.zip' '/content/drive/MyDrive/Colab Notebooks/models/fine-tuned-bert_v6_2.zip'